## Dakar Rally Scraper

This notebook provides a case study in retrieving, cleaning, organising and processing data obtained from  a third party website, specifically, timing and results data from the 2019 Dakar Rally.

Another way of thinking of it is as a series of marks created during an exploratory data analysis performance.

Or palaver.

Whatever.

Shall we begin?

In [1]:
#Use requests cache so we can keep an archive of the results HTML
import requests
import pandas as pd

from numpy import nan as NaN

In [2]:
import requests_cache
requests_cache.install_cache('dakar_cache', backend='sqlite',  expire_after=300)

Timing data is provided in two forms:

- time at waypoint / split;
- gap to leader at that waypoint;

Ranking data for the stage and overall at end of stage is also available.

Timing and ranking data is available for:

- car
- moto (motorbike)
- quad
- ssv
- truck


In [3]:
YEAR = 2020 #2019
STAGE = 1
VTYPE = 'car'

## Stage Info

Retrieve some basic information about a stage.

In [4]:
def get_stage_stats(stage):
    #stage_stats_url='https://gaps.dakar.com/2019/dakar/index_info.php?l=ukie&s={stage}&vh=a'
    stage_stats_url='https://gaps.dakar.com/2020/dakar/index_info.php?l=ukie&s={stage}&vh=a'

    html = requests.get(stage_stats_url.format(stage=STAGE)).content
        
    stage_stats_df=pd.read_html(html)[0]

    return stage_stats_df#.rename(columns=stage_stats_df.iloc[0]).drop(stage_stats_df.index[0])

In [5]:
get_stage_stats(STAGE)

,Special,Moto,Quad,Car,SSV,Truck
0,Start,07:20,08:34,09:20,10:49,11:50
1,Liaison,433km,433km,433km,433km,433km
2,Special,319km,319km,319km,319km,319km
3,Number of participants,Number of participants,Number of participants,Number of participants,Number of participants,Number of participants
4,At start,144,23,83,46,46
5,Left,144,23,82,46,41
6,Arrived,140,22,81,45,40
7,Latest WP,ass,ass,ass,ass,ass
8,Leader at latest WP,001 PRICE,250 CASALE,319 ZALA,419 DOMZALA,516 SHIBALOV
9,Nb at latest WP,140,22,81,45,40


## Timing Data

Typical of many rallies, the live timing pages return several sorts of data:

- times and gaps for each stage;
- stage and overall results for each stage.

In [6]:
URL_PATTERN='https://gaps.dakar.com/2019/dakar/?s={stage}&c=aso&l=ukie&vi={tab}&sv={timerank}&vh={vtype}&sws=99'
URL_PATTERN='https://gaps.dakar.com/2020/dakar/?s={stage}&c=aso&l=ukie&vi={tab}&sv={timerank}&vh={vtype}&sws=99'
#sws - selected waypoint?

In [7]:
#Vehicle types
VTYPE_ ={ 'car':'a','moto':'m','quad':'q','ssv':'s','truck':'c'}

#Screen / tab selection
TAB_ = {'timing':0,'news':1,'ranking':2}

#Options for timing data
TIMING_ = {'gap':0,'time':1}

#Options for ranking data
RANKING_ = {'stage':0, 'general':1}

## Previewing the data

Let's see what the data looks like...

*Uncomment and run the following to preview / inspect the data that's avaliable.*

In [8]:
#pd.read_html(URL_PATTERN.format(stage=STAGE,tab=TAB_['timing'],vtype=VTYPE_['car'],timerank='time'))

In [9]:
#pd.read_html(URL_PATTERN.format(stage=STAGE,tab=TAB_['ranking'],vtype=VTYPE_['car'],timerank='stage'))

By inspection, we note that:

- several tables are returned by each page;
- there are common identifier columns (`['Pos','Bib','Crew','Brand']`);
- there are irrelevant columns (`['View details','Select']`);
- the raw timing data columns (timig data at each waypoint) include information about split rank and how it compares to the rank at the previous split / waypoint; this needs to be cleaned before we can convert timestrings to timedeltas.

In [10]:
#TIMING = RANKING = 0 #deprecate these
TIME = RANK = 0
CREWTEAM = 1
BRANDS = 2
COUNTRIES = 3

## Retrieving the Data Tables

We can define helper functions to pull back tables associated with the timing or ranking pages.

In [11]:
#Retrieve a page

#Should we also return the stage so that tables stand alone as items with complete information?
def _data(stage,vtype='car',tab='timing', timerank='time', showstage=False):
    ''' Retrieve timing or ranking HTML page and scrape HTML tables. '''
    timerank = RANKING_[timerank] if tab=='ranking' else TIMING_[timerank]
        
    url = URL_PATTERN.format(stage=stage,tab=TAB_[tab],vtype=VTYPE_[vtype],timerank=timerank)
    html = requests.get(url).content
    _tmp = pd.read_html(html, na_values=['-'])
    if showstage:
        #This is a hack - elsewhere we use TIME, RANK etc in case there are more tables?
        _tmp[0].insert(0, 'Stage', stage)
    return _tmp


def _fetch_timing_data(stage,vtype='car', timerank='time', showstage=False):
    ''' Return data tables from timing page. '''
    _tmp = _data(stage,vtype=vtype, tab='timing', timerank=timerank, showstage=showstage)
    
    if 'View details' not in _tmp[TIME]:
        return []
    
    _tmp[TIME].drop(columns=['View details','Select'], inplace=True)
    return _tmp

def _fetch_ranking_data(stage,vtype='car', timerank='stage', showstage=False):
    ''' Return data tables from ranking page. '''
    rank_cols = ['Pos','Bib','Crew','Brand','Time','Gap','Penalty']
    _tmp = _data(stage,vtype=vtype, tab='ranking', timerank=timerank, showstage=showstage)
    
    if 'View details' not in _tmp[RANK]:
        return []
    
    _tmp[RANK].drop(columns=['View details','Select'], inplace=True)
    #if timerank=='general':
    #    _tmp[RANK].rename(columns={'Pos':'Overall Position'}, inplace=True)
    return _tmp


## Ranking Data

Process the ranking data...

So what have we got to work with?

In [12]:
rdata = _fetch_ranking_data(STAGE, vtype=VTYPE, timerank='general', showstage=True)

rdata[RANK].head()

,Stage,Pos,Bib,Crew,Brand,Time,Gap,Penalty
0,1,1,319,V. ZALA S. JURGELENAS AGRORODEO,MINI,03:19:04,0:00:00,00:00:00
1,1,2,302,S. PETERHANSEL P. FIUZA BAHRAIN JCW X-RAID TEAM,MINI,03:21:18,0:02:14,00:00:00
2,1,3,305,C. SAINZ L. CRUZ BAHRAIN JCW X-RAID TEAM,MINI,03:21:54,0:02:50,00:00:00
3,1,4,300,N. AL-ATTIYAH M. BAUMEL TOYOTA GAZOO RACING,TOYOTA,03:24:37,0:05:33,00:00:00
4,1,5,307,B. TEN BRINKE T. COLSOUL TOYOTA GAZOO RACING,TOYOTA,03:25:34,0:06:30,00:00:00


In [13]:
rdata[RANK].dtypes

Stage       int64
Pos         int64
Bib         int64
Crew       object
Brand      object
Time       object
Gap        object
Penalty    object
dtype: object

The basic retrieval returns a table with timing data as strings, and the `Bib` identifier as an integer.

The `Bib` identifer, We could also regard it as a string so that we aren't tempted to treat it as a number, inwhich case we should also ensure that any extraneous whitespace is stripped if the `Bib` was already a string:

```python
rdata[RANK]['Bib'] = rdata[RANK]['Bib'].astype(str).str.strip()
```

In [14]:
rdata[RANK]['Bib'] = rdata[RANK]['Bib'].astype(int)

## Convert time to timedelta

Several of the datasets return times, as strings, in the form: `HH:MM:SS`.

We can convert these times to timedeltas.

Timing related columns are all columns except those in `['Pos','Bib','Crew','Brand']`.

We can also prefix timing columns in the timing data screens so we can recreate the order they should appear in:

In [15]:
#Prefix each split designator with a split count
timingcols=['dss','wp1','wp2','ass']

In [16]:
{ x:'{}_{}'.format('{0:02d}'.format(i), x) for i, x in enumerate(timingcols, 0) }

{'dss': '00_dss', 'wp1': '01_wp1', 'wp2': '02_wp2', 'ass': '03_ass'}

One of the things we need to handle are timing columns where the timing data may be mixed with other sorts of data in the raw data table.

Routines for cleaner the data are included in the timing handler function but they were actually "backfilled" into the function after creating them (originally) later on in the notebook.

In [17]:
from pandas.api.types import is_string_dtype

#At first sight, this looks quite complicated, but a lot of it is backfilled 
# to take into account some of the cleaning we need to do for the full (messy) timing data
def _get_timing(df, typ=TIME, kind='simple'):
    ''' Convert times to time deltas and
        prefix waypoint / timing columns with a two digit counter. '''
    #kind: simple, full, raw
    #Some of the exclusion column names are backfilled into this function
    # from columns introduced later in the notebook
    # What we're trying to do is identify columns that aren't timing related
    timingcols = [c for c in df[typ].columns if c not in ['Pos','Overall Position','Bib','Crew','Brand',
                                                          'Refuel', 'Road Position', 'Stage'] ]
    
    #Clean up the data in a timing column, then cast to timedelta
    for col in timingcols:
        #A column of NAs may be incorrectly typed
        #print(df[typ].columns)
        df[typ][col] = df[typ][col].fillna('').str.strip()
        #In the simple approach, we just grab the timing data and dump the mess
        if kind!='full':
            df[typ][col] = df[typ][col].str.extract(r'(\d{1,2}:\d{1,2}:\d{1,2})')
        else:
            #The full on extractor - try to parse out all the data
            #  that has been munged into a timing column
            if col==timingcols[-1]:
                #There's an end effect:
                #  the last column in the timing dataset doesn't have position embedded in it
                # In this case, just pull out the position gained/maintained/lost flag
                df[typ][[col,col+'_gain']] = df[typ][col].str.extract(r'(\d{1,2}:\d{1,2}:\d{1,2})(.*)', expand=True)
            else:
                #In the main body of the table, position gain as well as waypoint rank position are available
                df[typ][[col,col+'_gain',col+'_pos']] = df[typ][col].str.extract(r'(\d{1,2}:\d{1,2}:\d{1,2})(.*)\((\d*)\)', expand=True)
                #Ideally, the pos cols would be of int type, but int doesn't support NA
                df[typ][col+'_pos'] = df[typ][col+'_pos'].astype(float)
        
        #Cast the time string to a timedelta
        if kind=='full' or kind=='raw':
            df[typ]['{}_raw'.format(col)] = df[typ][col][:]
        df[typ][col] = pd.to_timedelta( df[typ][col] )


    #In timing screen, rename cols with a leading two digit index
    #This allows us to report splits in order
    #We only want to do this for the timing data columns, not the rank timing columns...
    #Should really do this based on time type?
    timingcols = [c for c in timingcols if c not in ['Time','Gap','Penalty'] and not c.endswith(('_raw','_pos','_gain'))]
    timingcols_map = { x:'{}_{}'.format('{0:02d}'.format(i), x) for i, x in enumerate(timingcols, 0) }
    df[typ].rename(columns=timingcols_map, inplace=True)
    
    #TO_DO - need to number label the ..._raw, _pos and ..._gain cols
    #This is overly complex because it handles ranking and timing frames
    #Better to split out to separate ones?
    for suffix in ['_raw','_pos','_gain']:
        cols=[c for c in df[typ].columns if (not c.startswith(('Time','Gap','Penalty'))) and c.endswith(suffix)]
        cols_map = { x:'{}_{}'.format(x,'{0:02d}'.format(i)) for i, x in enumerate(cols, 0) }
        df[typ].rename(columns=cols_map, inplace=True)
#TO_DO - elsewhere: trap for start with 0/1 and not end _gain etc
                    
    return df


## Ranking Data Redux

Normalise the times as timedeltas.

For timestrings of the form `HH:MM:SS`, this is as simple as passing the timestring column to the *pandas* `.to_timedelta()` function:

```python
pd.to_timedelta( df[TIMESTRING_COLUMN] )
```

We just need to ensure we pass it the correct columns...

In [18]:
def get_ranking_data(stage,vtype='car', timerank='stage', kind='simple'):
    ''' Retrieve rank timing data and return it in a form we can work directly with. '''
    
    #kind: simple, raw
    df = _fetch_ranking_data(stage,vtype=vtype, timerank=timerank)
    df[RANK]['Bib'] = df[RANK]['Bib'].astype(int)
    return _get_timing(df, typ=RANK, kind=kind)

In [19]:
get_ranking_data(STAGE, VTYPE, kind='raw')[RANK].head()

,Pos,Bib,Crew,Brand,Time,Gap,Penalty,Time_raw,Gap_raw,Penalty_raw
0,1,319,V. ZALA S. JURGELENAS AGRORODEO,MINI,03:19:04,00:00:00,0 days,03:19:04,0:00:00,00:00:00
1,2,302,S. PETERHANSEL P. FIUZA BAHRAIN JCW X-RAID TEAM,MINI,03:21:18,00:02:14,0 days,03:21:18,0:02:14,00:00:00
2,3,305,C. SAINZ L. CRUZ BAHRAIN JCW X-RAID TEAM,MINI,03:21:54,00:02:50,0 days,03:21:54,0:02:50,00:00:00
3,4,300,N. AL-ATTIYAH M. BAUMEL TOYOTA GAZOO RACING,TOYOTA,03:24:37,00:05:33,0 days,03:24:37,0:05:33,00:00:00
4,5,307,B. TEN BRINKE T. COLSOUL TOYOTA GAZOO RACING,TOYOTA,03:25:34,00:06:30,0 days,03:25:34,0:06:30,00:00:00


In [20]:
STAGE, VTYPE

(1, 'car')

In [21]:
#What changed?
get_ranking_data(STAGE, VTYPE,timerank='general', kind='raw')[RANK].head()

,Pos,Bib,Crew,Brand,Time,Gap,Penalty,Time_raw,Gap_raw,Penalty_raw
0,1,319,V. ZALA S. JURGELENAS AGRORODEO,MINI,03:19:04,00:00:00,0 days,03:19:04,0:00:00,00:00:00
1,2,302,S. PETERHANSEL P. FIUZA BAHRAIN JCW X-RAID TEAM,MINI,03:21:18,00:02:14,0 days,03:21:18,0:02:14,00:00:00
2,3,305,C. SAINZ L. CRUZ BAHRAIN JCW X-RAID TEAM,MINI,03:21:54,00:02:50,0 days,03:21:54,0:02:50,00:00:00
3,4,300,N. AL-ATTIYAH M. BAUMEL TOYOTA GAZOO RACING,TOYOTA,03:24:37,00:05:33,0 days,03:24:37,0:05:33,00:00:00
4,5,307,B. TEN BRINKE T. COLSOUL TOYOTA GAZOO RACING,TOYOTA,03:25:34,00:06:30,0 days,03:25:34,0:06:30,00:00:00


In [22]:
ranking_data = get_ranking_data(STAGE, VTYPE)

ranking_data[RANK].head()

,Pos,Bib,Crew,Brand,Time,Gap,Penalty
0,1,319,V. ZALA S. JURGELENAS AGRORODEO,MINI,03:19:04,00:00:00,0 days
1,2,302,S. PETERHANSEL P. FIUZA BAHRAIN JCW X-RAID TEAM,MINI,03:21:18,00:02:14,0 days
2,3,305,C. SAINZ L. CRUZ BAHRAIN JCW X-RAID TEAM,MINI,03:21:54,00:02:50,0 days
3,4,300,N. AL-ATTIYAH M. BAUMEL TOYOTA GAZOO RACING,TOYOTA,03:24:37,00:05:33,0 days
4,5,307,B. TEN BRINKE T. COLSOUL TOYOTA GAZOO RACING,TOYOTA,03:25:34,00:06:30,0 days


In [23]:
ranking_data[RANK].dtypes

Pos                  int64
Bib                  int64
Crew                object
Brand               object
Time       timedelta64[ns]
Gap        timedelta64[ns]
Penalty    timedelta64[ns]
dtype: object

The `Crew` data is a bit of a mishmash. If we were to normalise this table, we'd have to split that data out...

For now, let's leave it...

...because sometimes, it can be handy to be able to pull out a chunk of unnormalised data as a simple string.

In [24]:
ranking_data[RANK].dtypes

Pos                  int64
Bib                  int64
Crew                object
Brand               object
Time       timedelta64[ns]
Gap        timedelta64[ns]
Penalty    timedelta64[ns]
dtype: object

## Timing Data

The timing data needs some processing:

In [25]:
data = _fetch_timing_data(STAGE, VTYPE)

data[TIME][60:70]

,Pos,Bib,Crew,Brand,dss,km47,km86,km105,km158,km208,km254,km294,ass
60,61.0,368,G. PETRUS T. JANCYS PETRUS RACING,PETRUS RACING,10:24:00= (66),001:14:45▼ (67),001:55:48▲ (64),NaN,002:55:14= (64),003:35:49= (64),004:21:25▲ (61),005:07:23= (61),005:25:49=
61,62.0,344,B. PATTON R. PIERCE OVERDRIVE TOYOTA,OVERDRIVE TOYOTA,10:12:30= (43),001:25:03▼ (72),002:13:22▲ (69),NaN,003:09:16▲ (67),003:51:24▼ (69),004:33:02▲ (67),005:25:27▲ (65),005:43:01▲
62,63.0,373,Y. ZHAO K. YAN QIAN’AN JIU JIANG LANDSAIL RACI...,QIAN’AN JIU JIANG LANDSAIL RACING CLUB,10:25:47= (70),001:03:08▲ (63),002:07:17▼ (67),NaN,002:55:18▲ (65),003:34:57▲ (63),004:24:04▲ (62),005:12:37= (62),005:45:15▼
63,64.0,369,T. STAM R. BARGEMAN SCHIJF RALLY,SCHIJF RALLY,10:24:30= (67),001:02:49▲ (62),002:20:26▼ (71),NaN,003:10:12▲ (68),003:51:07= (68),004:41:29▼ (69),005:28:24▲ (66),005:46:14▲
64,65.0,346,G. TRAMONI D. TOTAIN TEAM 100% SUD OUEST,TEAM 100% SUD OUEST,10:13:30= (45),000:59:44▼ (58),001:59:12▼ (66),NaN,003:02:21= (66),003:46:39▼ (67),004:29:55▲ (64),005:36:15▼ (69),005:56:40▲
65,66.0,353,JC. VALLEJO L. BARONIO TEAM PRORAID PERU,TEAM PRORAID PERU,10:17:00= (52),000:54:46▼ (54),001:28:54▲ (50),NaN,002:26:32▼ (51),003:33:00▼ (62),004:36:29▼ (68),005:31:42▲ (67),005:57:09▲
66,67.0,383,S. ALSHUMMARI SN. ALTAMIMI SALMAN AL SHUMMARI ...,SALMAN AL SHUMMARI TEAM,10:29:30= (76),001:01:58▲ (61),001:39:37▲ (58),NaN,002:43:12▼ (60),003:24:22▲ (59),004:28:23▼ (63),005:35:35▼ (68),006:00:54▲
67,68.0,357,F. TUHEIL P. TUHEIL TEAM 100% SUD OUEST,TEAM 100% SUD OUEST,10:19:00= (55),001:18:09▼ (69),002:16:33▼ (70),NaN,002:54:19▲ (62),003:39:48▼ (65),004:31:30▼ (66),005:23:16▲ (64),006:06:16▼
68,69.0,376,T. RICHARD F. MALDONADO SODICARS RACING,SODICARS RACING,10:27:00= (71),001:21:51▲ (70),002:25:08▼ (74),NaN,003:29:37▲ (69),004:15:11▼ (70),005:07:34= (70),006:01:40= (70),006:20:40▲
69,70.0,361,M. ALTWIJRI K. ALMARSHOOD ALTUWAIJRI RACING TEAM,ALTUWAIJRI RACING TEAM,10:21:00= (60),NaN,001:28:55▲ (51),NaN,002:31:35▼ (54),003:44:07▼ (66),004:31:26▲ (65),005:21:01▲ (63),006:37:34▼


A full inspection of the time data shows that some additional metadata corresponding to whether in-stage refuelling is allowed may also be recorded in the `Bib` column (for example, `403 ⛽`).

We can extract this information into a separate dataframe / table.

In [26]:
def get_refuel_status(df):
    ''' Parse the refuel status out of timing data Bib column.
        Return extended dataframe with a clean Bib column and a new Refuel column. '''
    
    #The .str.extract() function allows us to match separate groups using a regex
    #  and return the corresponding group data as distinct columns
    #Force the Bin type to a str if it isn't created as such so we can regex it...
    df[['Bib','_tmp']] = df['Bib'].astype(str).str.extract(r'(\d*)([^\d]*)', expand=True)
    
    #Set the Refuel status as a Boolean
    df.insert(2, 'Refuel', df['_tmp'])
    df.drop('_tmp', axis=1, inplace=True)
    df['Refuel'] = df['Refuel']!=''
    
    #Set the Bib value as an int
    df['Bib'] = df['Bib'].astype(int)
    
    return df

In [27]:
data[TIME] = get_refuel_status(data[TIME])
data[TIME][60:70]

,Pos,Bib,Refuel,Crew,Brand,dss,km47,km86,km105,km158,km208,km254,km294,ass
60,61.0,368,False,G. PETRUS T. JANCYS PETRUS RACING,PETRUS RACING,10:24:00= (66),001:14:45▼ (67),001:55:48▲ (64),NaN,002:55:14= (64),003:35:49= (64),004:21:25▲ (61),005:07:23= (61),005:25:49=
61,62.0,344,False,B. PATTON R. PIERCE OVERDRIVE TOYOTA,OVERDRIVE TOYOTA,10:12:30= (43),001:25:03▼ (72),002:13:22▲ (69),NaN,003:09:16▲ (67),003:51:24▼ (69),004:33:02▲ (67),005:25:27▲ (65),005:43:01▲
62,63.0,373,False,Y. ZHAO K. YAN QIAN’AN JIU JIANG LANDSAIL RACI...,QIAN’AN JIU JIANG LANDSAIL RACING CLUB,10:25:47= (70),001:03:08▲ (63),002:07:17▼ (67),NaN,002:55:18▲ (65),003:34:57▲ (63),004:24:04▲ (62),005:12:37= (62),005:45:15▼
63,64.0,369,False,T. STAM R. BARGEMAN SCHIJF RALLY,SCHIJF RALLY,10:24:30= (67),001:02:49▲ (62),002:20:26▼ (71),NaN,003:10:12▲ (68),003:51:07= (68),004:41:29▼ (69),005:28:24▲ (66),005:46:14▲
64,65.0,346,False,G. TRAMONI D. TOTAIN TEAM 100% SUD OUEST,TEAM 100% SUD OUEST,10:13:30= (45),000:59:44▼ (58),001:59:12▼ (66),NaN,003:02:21= (66),003:46:39▼ (67),004:29:55▲ (64),005:36:15▼ (69),005:56:40▲
65,66.0,353,False,JC. VALLEJO L. BARONIO TEAM PRORAID PERU,TEAM PRORAID PERU,10:17:00= (52),000:54:46▼ (54),001:28:54▲ (50),NaN,002:26:32▼ (51),003:33:00▼ (62),004:36:29▼ (68),005:31:42▲ (67),005:57:09▲
66,67.0,383,False,S. ALSHUMMARI SN. ALTAMIMI SALMAN AL SHUMMARI ...,SALMAN AL SHUMMARI TEAM,10:29:30= (76),001:01:58▲ (61),001:39:37▲ (58),NaN,002:43:12▼ (60),003:24:22▲ (59),004:28:23▼ (63),005:35:35▼ (68),006:00:54▲
67,68.0,357,False,F. TUHEIL P. TUHEIL TEAM 100% SUD OUEST,TEAM 100% SUD OUEST,10:19:00= (55),001:18:09▼ (69),002:16:33▼ (70),NaN,002:54:19▲ (62),003:39:48▼ (65),004:31:30▼ (66),005:23:16▲ (64),006:06:16▼
68,69.0,376,False,T. RICHARD F. MALDONADO SODICARS RACING,SODICARS RACING,10:27:00= (71),001:21:51▲ (70),002:25:08▼ (74),NaN,003:29:37▲ (69),004:15:11▼ (70),005:07:34= (70),006:01:40= (70),006:20:40▲
69,70.0,361,False,M. ALTWIJRI K. ALMARSHOOD ALTUWAIJRI RACING TEAM,ALTUWAIJRI RACING TEAM,10:21:00= (60),NaN,001:28:55▲ (51),NaN,002:31:35▼ (54),003:44:07▼ (66),004:31:26▲ (65),005:21:01▲ (63),006:37:34▼


We also notice that the raw timing data includes information about split rank and how it compares to the rank at the previous split / waypoint, with the raw data taking the form `08:44:00= (11)`. Which is to say, `HH:MM:DDx (NN?)` where `x` is a comparator showing whether the rank at that waypoint improved (▲), remained the same as (=), or worsened (▼) compared to the previous waypoint.

Note that the final `ass` column does not include the rank.

We can use a regular expression to separate the data out, with each regex group being expanded into a separate column:

In [28]:
data[TIME]['dss'].str.extract(r'(\d{2}:\d{2}:\d{2})(.*)\((\d*)\)', expand=True).head()

,0,1,2
0,10:00:00,=,19
1,09:26:00,=,3
2,09:35:00,=,6
3,09:20:00,=,1
4,09:41:00,=,8


We can backfill an expression of that form into the timing data handler function above...

Now we wrap several steps together into a function that gets us a clean set of timing data, with columns of an appropriate type:

In [29]:
def get_timing_data(stage,vtype='car', timerank='time', kind='simple'):
    ''' Get timing data in a form ready to use. '''
    df = _fetch_timing_data(stage,vtype=vtype, timerank=timerank)
    
    if not df:
        return []
    
    df[TIME] = get_refuel_status(df[TIME])
    return _get_timing(df, typ=TIME, kind=kind)

In [30]:
get_timing_data(STAGE, VTYPE, kind='simple')[TIME].head()

,Pos,Bib,Refuel,Crew,Brand,00_dss,01_km47,02_km86,03_km105,04_km158,05_km208,06_km254,07_km294,08_ass
0,1.0,319,False,V. ZALA S. JURGELENAS AGRORODEO,AGRORODEO,10:00:00,00:36:18,01:02:42,NaT,01:37:33,02:06:34,02:36:17,03:06:26,03:19:04
1,2.0,302,False,S. PETERHANSEL P. FIUZA BAHRAIN JCW X-RAID TEAM,BAHRAIN JCW X-RAID TEAM,09:26:00,00:35:30,01:00:34,01:14:30,01:35:05,02:08:07,02:37:39,03:08:30,03:21:18
2,3.0,305,False,C. SAINZ L. CRUZ BAHRAIN JCW X-RAID TEAM,BAHRAIN JCW X-RAID TEAM,09:35:00,00:34:43,00:59:53,01:13:38,01:44:51,02:11:52,02:40:45,03:10:07,03:21:54
3,4.0,300,False,N. AL-ATTIYAH M. BAUMEL TOYOTA GAZOO RACING,TOYOTA GAZOO RACING,09:20:00,00:34:00,00:58:41,01:11:20,01:33:55,02:01:51,02:37:52,03:11:30,03:24:37
4,5.0,307,False,B. TEN BRINKE T. COLSOUL TOYOTA GAZOO RACING,TOYOTA GAZOO RACING,09:41:00,00:36:01,01:02:28,01:15:44,01:39:18,02:08:17,02:41:23,03:12:44,03:25:34


In [31]:
data = get_timing_data(STAGE, VTYPE, kind='full')
data[TIME].head()

,Pos,Bib,Refuel,Crew,Brand,00_dss,01_km47,02_km86,03_km105,04_km158,...,km208_pos_05,km208_raw_05,km254_gain_06,km254_pos_06,km254_raw_06,km294_gain_07,km294_pos_07,km294_raw_07,ass_gain_08,ass_raw_08
0,1.0,319,False,V. ZALA S. JURGELENAS AGRORODEO,AGRORODEO,10:00:00,00:36:18,01:02:42,NaT,01:37:33,...,3.0,02:06:34,▲,2.0,02:36:17,▲,1.0,03:06:26,=,03:19:04
1,2.0,302,False,S. PETERHANSEL P. FIUZA BAHRAIN JCW X-RAID TEAM,BAHRAIN JCW X-RAID TEAM,09:26:00,00:35:30,01:00:34,01:14:30,01:35:05,...,4.0,02:08:07,▲,3.0,02:37:39,▲,2.0,03:08:30,=,03:21:18
2,3.0,305,False,C. SAINZ L. CRUZ BAHRAIN JCW X-RAID TEAM,BAHRAIN JCW X-RAID TEAM,09:35:00,00:34:43,00:59:53,01:13:38,01:44:51,...,9.0,02:11:52,▲,6.0,02:40:45,▲,3.0,03:10:07,=,03:21:54
3,4.0,300,False,N. AL-ATTIYAH M. BAUMEL TOYOTA GAZOO RACING,TOYOTA GAZOO RACING,09:20:00,00:34:00,00:58:41,01:11:20,01:33:55,...,1.0,02:01:51,▼,4.0,02:37:52,▼,5.0,03:11:30,▲,03:24:37
4,5.0,307,False,B. TEN BRINKE T. COLSOUL TOYOTA GAZOO RACING,TOYOTA GAZOO RACING,09:41:00,00:36:01,01:02:28,01:15:44,01:39:18,...,5.0,02:08:17,▼,8.0,02:41:23,▲,6.0,03:12:44,▲,03:25:34


In [32]:
data[TIME].dtypes

Pos                      float64
Bib                        int64
Refuel                      bool
Crew                      object
Brand                     object
00_dss           timedelta64[ns]
01_km47          timedelta64[ns]
02_km86          timedelta64[ns]
03_km105         timedelta64[ns]
04_km158         timedelta64[ns]
05_km208         timedelta64[ns]
06_km254         timedelta64[ns]
07_km294         timedelta64[ns]
08_ass           timedelta64[ns]
dss_gain_00               object
dss_pos_00               float64
dss_raw_00                object
km47_gain_01              object
km47_pos_01              float64
km47_raw_01               object
km86_gain_02              object
km86_pos_02              float64
km86_raw_02               object
km105_gain_03             object
km105_pos_03             float64
km105_raw_03              object
km158_gain_04             object
km158_pos_04             float64
km158_raw_04              object
km208_gain_05             object
km208_pos_

## Parse Metadata

Some of the scraped tables are used to provide selection lists, but we might be able to use them as metadata tables.

For example, here's a pretty complete set, although mangled together, set of competititor names, nationalities, and team names:

In [33]:
data[ CREWTEAM ].head()

,Highlight,Filter,Bib,Names
0,Highglight crew,Filter crew,1,T. PRICE (Australia)RED BULL KTM FACTORY TEAM
1,Highglight crew,Filter crew,2,M. WALKNER (Austria)RED BULL KTM FACTORY TEAM
2,Highglight crew,Filter crew,3,S. SUNDERLAND (United Kingdom)RED BULL KTM FAC...
3,Highglight crew,Filter crew,4,A. VAN BEVEREN (France)MONSTER ENERGY YAMAHA R...
4,Highglight crew,Filter crew,5,P. QUINTANILLA (Chile)ROCKSTAR ENERGY HUSQVARN...


It'll probably be convenient to have the unique `Bib` values available as an index:

In [34]:
data[ CREWTEAM ] = data[ CREWTEAM ][['Bib', 'Names']].set_index('Bib')
data[ CREWTEAM ].head()

,Names
Bib,
1,T. PRICE (Australia)RED BULL KTM FACTORY TEAM
2,M. WALKNER (Austria)RED BULL KTM FACTORY TEAM
3,S. SUNDERLAND (United Kingdom)RED BULL KTM FAC...
4,A. VAN BEVEREN (France)MONSTER ENERGY YAMAHA R...
5,P. QUINTANILLA (Chile)ROCKSTAR ENERGY HUSQVARN...


The `Names` may have several `Name (Country)` values, followed by a team name. The original HTML uses `<span>` tags to separate out values but the *pandas* `.read_html()` function flattens cell contents.
    
Let's have a go at pulling out the team names, which appear at the end of the string. If we can split each name, and the team name, into separate columns, and then metl those columns into separate rows, grouped by `Bib` number, we should be able to grab the last row, corrsponding to the team, in each group:

In [35]:
#Perhaps split on brackets?
# At least one team has brackets in the name at the end of the name
# So let's make that case, at least, a "not bracket" by setting a ) at the end to a :]:
# so we don't (mistakenly) split on it as if it were a country-associated bracket.
teams = data[ CREWTEAM ]['Names'].str.replace(r'\)$',':]:').str.split(')').apply(pd.Series).reset_index().melt(id_vars='Bib', var_name='Num').dropna()

#Find last item in each group, which is to say: the team
teamnames = teams.groupby('Bib').last()
#Defudge any brackets at the end back
teamnames = teamnames['value'].str.replace(':]:',')')
teamnames.head()

Bib
1                   RED BULL KTM FACTORY TEAM
2                   RED BULL KTM FACTORY TEAM
3                   RED BULL KTM FACTORY TEAM
4            MONSTER ENERGY YAMAHA RALLY TEAM
5    ROCKSTAR ENERGY HUSQVARNA FACTORY RACING
Name: value, dtype: object

Now let's go after the competitors. These are all *but* the last row in each group:

In [36]:
#Remove last row in group i.e. the team
personnel = teams.groupby('Bib').apply(lambda x: x.iloc[:-1]).set_index('Bib').reset_index()

personnel[['Name','Country']] = personnel['value'].str.split('(').apply(pd.Series)

#Strip whitespace
for c in ['Name','Country']:
    personnel[c] = personnel[c].str.strip()
    
personnel[['Bib','Num','Name','Country']].head()

,Bib,Num,Name,Country
0,1,0,T. PRICE,Australia
1,2,0,M. WALKNER,Austria
2,3,0,S. SUNDERLAND,United Kingdom
3,4,0,A. VAN BEVEREN,France
4,5,0,P. QUINTANILLA,Chile


For convenience, we might want to reshape this long form back to a wide form, with a single string containing all the competitor names associated with a particular `Bib` identifier:

In [37]:
#Create a single name string for each vehicle
#For each Bib number, group the rows associated with that number
# and aggregate the names in those rows into a single, comma separated, joined string
# indexed by the corresponding Bib number
personnel.groupby('Bib')['Name'].agg(lambda col: ', '.join(col)).tail()

Bib
546    J. GINESTA, M. DARDAILLON, A. LOUREIRO
547          S. BESNARD, A. VITSE, S. LALICHE
548    D. INGELS, JB. CASSOULET, J. SCHOTANUS
549       A. BENBEKHTI, R. OSMANI, B. SEILLET
550     D. BERGHMANS, L. LORENZATO, T. GEUENS
Name: Name, dtype: object

In [38]:
data[ BRANDS ].head()

,Filter,Names
0,Filter brand,NaN
1,Filter brand,2WD
2,Filter brand,BETA
3,Filter brand,BMW
4,Filter brand,BORGWARD


In [39]:
data[ COUNTRIES ].head()

,Filter,Names
0,Filter country,Andorra (AND)
1,Filter country,United Arab Emirates (ARE)
2,Filter country,Argentina (ARG)
3,Filter country,Australia (AUS)
4,Filter country,Austria (AUT)


In [40]:
data[ COUNTRIES ][['Country','CountryCode']] = data[ COUNTRIES ]['Names'].str.extract(r'(.*) \((.*)\)',expand=True)
data[ COUNTRIES ].head()

,Filter,Names,Country,CountryCode
0,Filter country,Andorra (AND),Andorra,AND
1,Filter country,United Arab Emirates (ARE),United Arab Emirates,ARE
2,Filter country,Argentina (ARG),Argentina,ARG
3,Filter country,Australia (AUS),Australia,AUS
4,Filter country,Austria (AUT),Austria,AUT


If we have a col with a lot of missing timing data, should we drop it early? A downside of this is we are working with live data, there is likely to be lots of missing data so we can't run live tables? The following `get_annotated_timing_data()` definition takes the *delete cols with missing data* approach...

In [41]:
def get_annotated_timing_data(stage, vtype='car',
                              timerank='time', kind='simple', MAXMISSING=10):
    ''' Return a timing dataset that's ready to use. '''
    
    df = get_timing_data(stage, vtype, timerank, kind)
    if not df:
        return []
    
    #TEST
    df[TIME].dropna(thresh=MAXMISSING,axis=1,inplace=True)
    
    col00 = [c for c in df[TIME].columns if c.startswith('00_')][0]
    
    #Add in Road position
    df[TIME].insert(2,'Road Position', df[TIME].sort_values(col00,ascending=True)[col00].rank())
    
    return df

In [42]:
get_annotated_timing_data(STAGE,vtype=VTYPE, timerank='time', kind='full')[TIME].head()

,Pos,Bib,Road Position,Refuel,Crew,Brand,00_dss,01_km47,02_km86,04_km158,...,km208_pos_05,km208_raw_05,km254_gain_06,km254_pos_06,km254_raw_06,km294_gain_07,km294_pos_07,km294_raw_07,ass_gain_08,ass_raw_08
0,1.0,319,19.0,False,V. ZALA S. JURGELENAS AGRORODEO,AGRORODEO,10:00:00,00:36:18,01:02:42,01:37:33,...,3.0,02:06:34,▲,2.0,02:36:17,▲,1.0,03:06:26,=,03:19:04
1,2.0,302,3.0,False,S. PETERHANSEL P. FIUZA BAHRAIN JCW X-RAID TEAM,BAHRAIN JCW X-RAID TEAM,09:26:00,00:35:30,01:00:34,01:35:05,...,4.0,02:08:07,▲,3.0,02:37:39,▲,2.0,03:08:30,=,03:21:18
2,3.0,305,6.0,False,C. SAINZ L. CRUZ BAHRAIN JCW X-RAID TEAM,BAHRAIN JCW X-RAID TEAM,09:35:00,00:34:43,00:59:53,01:44:51,...,9.0,02:11:52,▲,6.0,02:40:45,▲,3.0,03:10:07,=,03:21:54
3,4.0,300,1.0,False,N. AL-ATTIYAH M. BAUMEL TOYOTA GAZOO RACING,TOYOTA GAZOO RACING,09:20:00,00:34:00,00:58:41,01:33:55,...,1.0,02:01:51,▼,4.0,02:37:52,▼,5.0,03:11:30,▲,03:24:37
4,5.0,307,8.0,False,B. TEN BRINKE T. COLSOUL TOYOTA GAZOO RACING,TOYOTA GAZOO RACING,09:41:00,00:36:01,01:02:28,01:39:18,...,5.0,02:08:17,▼,8.0,02:41:23,▲,6.0,03:12:44,▲,03:25:34


In [43]:
t_data = get_annotated_timing_data(STAGE,vtype=VTYPE, timerank='time')[TIME]
t_data.head(10)

,Pos,Bib,Road Position,Refuel,Crew,Brand,00_dss,01_km47,02_km86,04_km158,05_km208,06_km254,07_km294,08_ass
0,1.0,319,19.0,False,V. ZALA S. JURGELENAS AGRORODEO,AGRORODEO,10:00:00,00:36:18,01:02:42,01:37:33,02:06:34,02:36:17,03:06:26,03:19:04
1,2.0,302,3.0,False,S. PETERHANSEL P. FIUZA BAHRAIN JCW X-RAID TEAM,BAHRAIN JCW X-RAID TEAM,09:26:00,00:35:30,01:00:34,01:35:05,02:08:07,02:37:39,03:08:30,03:21:18
2,3.0,305,6.0,False,C. SAINZ L. CRUZ BAHRAIN JCW X-RAID TEAM,BAHRAIN JCW X-RAID TEAM,09:35:00,00:34:43,00:59:53,01:44:51,02:11:52,02:40:45,03:10:07,03:21:54
3,4.0,300,1.0,False,N. AL-ATTIYAH M. BAUMEL TOYOTA GAZOO RACING,TOYOTA GAZOO RACING,09:20:00,00:34:00,00:58:41,01:33:55,02:01:51,02:37:52,03:11:30,03:24:37
4,5.0,307,8.0,False,B. TEN BRINKE T. COLSOUL TOYOTA GAZOO RACING,TOYOTA GAZOO RACING,09:41:00,00:36:01,01:02:28,01:39:18,02:08:17,02:41:23,03:12:44,03:25:34
5,6.0,311,12.0,False,O. TERRANOVA B. GRAUE X-RAID MINI JCW TEAM,X-RAID MINI JCW TEAM,09:53:00,00:34:40,01:01:39,01:37:11,02:12:09,02:40:39,03:11:20,03:26:19
6,7.0,315,15.0,False,M. SERRADORI F. LURQUIN SRT RACING,SRT RACING,09:56:00,00:34:55,01:00:21,01:39:01,02:10:20,02:40:58,03:12:57,03:27:59
7,8.0,317,17.0,False,V. VASILYEV V. YEVTYEKHOV X-RAID G-ENERGY,X-RAID G-ENERGY,09:58:00,00:38:23,01:06:49,01:43:37,02:11:41,02:41:49,03:18:05,03:32:29
8,9.0,309,10.0,False,Y. AL RAJHI K. ZHILTSOV OVERDRIVE TOYOTA,OVERDRIVE TOYOTA,09:47:00,00:44:47,01:09:33,01:43:36,02:15:27,02:44:42,03:18:53,03:32:50
9,10.0,314,14.0,False,E. VAN LOON S. DELAUNAY OVERDRIVE TOYOTA,OVERDRIVE TOYOTA,09:55:00,00:40:42,01:07:58,01:44:54,02:13:47,02:46:37,03:20:02,03:33:02


In [44]:
not_timing_cols = ['Pos','Road Position','Refuel','Bib','Crew','Brand']

driver_data = t_data[ not_timing_cols ]
driver_data.head()

,Pos,Road Position,Refuel,Bib,Crew,Brand
0,1.0,19.0,False,319,V. ZALA S. JURGELENAS AGRORODEO,AGRORODEO
1,2.0,3.0,False,302,S. PETERHANSEL P. FIUZA BAHRAIN JCW X-RAID TEAM,BAHRAIN JCW X-RAID TEAM
2,3.0,6.0,False,305,C. SAINZ L. CRUZ BAHRAIN JCW X-RAID TEAM,BAHRAIN JCW X-RAID TEAM
3,4.0,1.0,False,300,N. AL-ATTIYAH M. BAUMEL TOYOTA GAZOO RACING,TOYOTA GAZOO RACING
4,5.0,8.0,False,307,B. TEN BRINKE T. COLSOUL TOYOTA GAZOO RACING,TOYOTA GAZOO RACING


In [45]:
def get_driver_data(stage, topN=None, vtype='car',):
    driver_data = get_annotated_timing_data(stage, vtype=vtype,
                                            timerank='time')[TIME]

    driver_data = driver_data[['Bib','Pos','Road Position','Crew','Brand']]
    driver_data.set_index('Bib', inplace=True)
    if topN:
        return driver_data[(driver_data['Pos']<=topN)]
    
    return driver_data

In [46]:
get_driver_data(STAGE)

,Pos,Road Position,Crew,Brand
Bib,,,,
319,1.0,19.0,V. ZALA S. JURGELENAS AGRORODEO,AGRORODEO
302,2.0,3.0,S. PETERHANSEL P. FIUZA BAHRAIN JCW X-RAID TEAM,BAHRAIN JCW X-RAID TEAM
305,3.0,6.0,C. SAINZ L. CRUZ BAHRAIN JCW X-RAID TEAM,BAHRAIN JCW X-RAID TEAM
300,4.0,1.0,N. AL-ATTIYAH M. BAUMEL TOYOTA GAZOO RACING,TOYOTA GAZOO RACING
307,5.0,8.0,B. TEN BRINKE T. COLSOUL TOYOTA GAZOO RACING,TOYOTA GAZOO RACING
...,...,...,...,...
377,79.0,72.0,J. FONT B. RODRIGUEZ FN SPEED TEAM,FN SPEED TEAM
348,80.0,47.0,B. ZAPAG JJ. SANCHEZ SOUTH RACING,SOUTH RACING
374,81.0,NaN,A. ALYAEESH FM. FTYH NAIZK,NAIZK


The number of waypoints differs across stages. If we cast the wide format waypoint data into a long form, we can more conveniently merge waypoint timing data from separate stages into the same dataframe.

In [47]:
pd.melt(t_data.head(),
        id_vars=not_timing_cols,
        var_name='Waypoint', value_name='Time').head()

,Pos,Road Position,Refuel,Bib,Crew,Brand,Waypoint,Time
0,1.0,19.0,False,319,V. ZALA S. JURGELENAS AGRORODEO,AGRORODEO,00_dss,10:00:00
1,2.0,3.0,False,302,S. PETERHANSEL P. FIUZA BAHRAIN JCW X-RAID TEAM,BAHRAIN JCW X-RAID TEAM,00_dss,09:26:00
2,3.0,6.0,False,305,C. SAINZ L. CRUZ BAHRAIN JCW X-RAID TEAM,BAHRAIN JCW X-RAID TEAM,00_dss,09:35:00
3,4.0,1.0,False,300,N. AL-ATTIYAH M. BAUMEL TOYOTA GAZOO RACING,TOYOTA GAZOO RACING,00_dss,09:20:00
4,5.0,8.0,False,307,B. TEN BRINKE T. COLSOUL TOYOTA GAZOO RACING,TOYOTA GAZOO RACING,00_dss,09:41:00


In [48]:
def _timing_long(df, nodss=True):
    ''' Cast timing data to long data frame. '''
    df = pd.melt(df,
                 id_vars=[c for c in df.columns if not any(_c in c for _c in ['dss','wp','ass', 'km', 'pk'])],
                 var_name='Waypoint', value_name='Time')

    if nodss:
        return df[~df['Waypoint'].str.startswith('00')]
    return df
#Should return cols: Pos	Bib	Road Position	Refuel	Crew	Brand	Waypoint	Time


def _typ_long(df, typ='_pos_', nodss=False):
    ''' Cast wide data to long data frame. '''
    
    df = pd.melt(df[['Bib']+[c for c in df.columns if typ in c]],
                 id_vars=['Bib'],
                 var_name='Waypoint', value_name=typ)
    df['WaypointOrder'] = df['Waypoint'].str.slice(-2).astype(int)
    if nodss:
        return df[~df['Waypoint'].str.startswith('00')]
    return df
#Should return cols: Bib	Waypoint	_gain_	Waypoint_idx

In [49]:
t_data_long = _timing_long(t_data)
t_data_long.head()

,Pos,Bib,Road Position,Refuel,Crew,Brand,Waypoint,Time
83,1.0,319,19.0,False,V. ZALA S. JURGELENAS AGRORODEO,AGRORODEO,01_km47,00:36:18
84,2.0,302,3.0,False,S. PETERHANSEL P. FIUZA BAHRAIN JCW X-RAID TEAM,BAHRAIN JCW X-RAID TEAM,01_km47,00:35:30
85,3.0,305,6.0,False,C. SAINZ L. CRUZ BAHRAIN JCW X-RAID TEAM,BAHRAIN JCW X-RAID TEAM,01_km47,00:34:43
86,4.0,300,1.0,False,N. AL-ATTIYAH M. BAUMEL TOYOTA GAZOO RACING,TOYOTA GAZOO RACING,01_km47,00:34:00
87,5.0,307,8.0,False,B. TEN BRINKE T. COLSOUL TOYOTA GAZOO RACING,TOYOTA GAZOO RACING,01_km47,00:36:01


In [50]:
t_data2 = get_annotated_timing_data(STAGE,vtype=VTYPE, timerank='time', kind='full')[TIME]
display(_typ_long(t_data2, '_gain_').head())
#_pos_, _raw_, _gain_       
#Clear down
t_data2 = None

,Bib,Waypoint,_gain_,WaypointOrder
0,319,dss_gain_00,=,0
1,302,dss_gain_00,=,0
2,305,dss_gain_00,=,0
3,300,dss_gain_00,=,0
4,307,dss_gain_00,=,0


In [51]:
def get_long_annotated_timing_data(stage, vtype='car', timerank='time', kind='simple'):
    ''' Get annotated timing dataframe and convert it to long format. '''
    
    #TO DO: But for the db, we want the raw long, not the time long
    _tmp = get_annotated_timing_data(stage, vtype, timerank, kind=kind)
    if not _tmp:
        return []
    
    #I don't think this works for anything other than kind=simple
    #TO DO: do we need to cope with the kind='full' stuff differently?
    _tmp[TIME] = _timing_long(_tmp[TIME])
    if kind=='simple':
        #Should really be testing if starts with an int
        _tmp[TIME]=_tmp[TIME][_tmp[TIME]['Waypoint'].str.startswith(('0','1'))]
    
    #Find the total seconds for each split / waypoint duration
    _tmp[TIME]['TimeInS'] = _tmp[TIME]['Time'].dt.total_seconds()
    
    if timerank=='gap':
        _tmp[TIME].rename(columns={'Time':'Gap', 'TimeInS':'GapInS'}, inplace=True)
    
    
    return _tmp

#Should return cols:
#Pos	Bib	Road Position	Refuel	Crew	Brand	Section	Gap	GapInS

In [52]:
#get_long_annotated_timing_data(3,vtype='quad', timerank='time')[TIME]

In [53]:
get_long_annotated_timing_data(STAGE, VTYPE)[TIME].head()

,Pos,Bib,Road Position,Refuel,Crew,Brand,Waypoint,Time,TimeInS
83,1.0,319,19.0,False,V. ZALA S. JURGELENAS AGRORODEO,AGRORODEO,01_km47,00:36:18,2178.0
84,2.0,302,3.0,False,S. PETERHANSEL P. FIUZA BAHRAIN JCW X-RAID TEAM,BAHRAIN JCW X-RAID TEAM,01_km47,00:35:30,2130.0
85,3.0,305,6.0,False,C. SAINZ L. CRUZ BAHRAIN JCW X-RAID TEAM,BAHRAIN JCW X-RAID TEAM,01_km47,00:34:43,2083.0
86,4.0,300,1.0,False,N. AL-ATTIYAH M. BAUMEL TOYOTA GAZOO RACING,TOYOTA GAZOO RACING,01_km47,00:34:00,2040.0
87,5.0,307,8.0,False,B. TEN BRINKE T. COLSOUL TOYOTA GAZOO RACING,TOYOTA GAZOO RACING,01_km47,00:36:01,2161.0


In [54]:
get_long_annotated_timing_data(STAGE, VTYPE, 'gap')[TIME].head()

,Pos,Bib,Road Position,Refuel,Crew,Brand,Waypoint,Gap,GapInS
83,1.0,319,19.0,False,V. ZALA S. JURGELENAS AGRORODEO,AGRORODEO,01_km47,00:02:18,138.0
84,2.0,302,3.0,False,S. PETERHANSEL P. FIUZA BAHRAIN JCW X-RAID TEAM,BAHRAIN JCW X-RAID TEAM,01_km47,00:01:30,90.0
85,3.0,305,6.0,False,C. SAINZ L. CRUZ BAHRAIN JCW X-RAID TEAM,BAHRAIN JCW X-RAID TEAM,01_km47,00:00:43,43.0
86,4.0,300,1.0,False,N. AL-ATTIYAH M. BAUMEL TOYOTA GAZOO RACING,TOYOTA GAZOO RACING,01_km47,00:00:00,0.0
87,5.0,307,8.0,False,B. TEN BRINKE T. COLSOUL TOYOTA GAZOO RACING,TOYOTA GAZOO RACING,01_km47,00:02:01,121.0


## Find the time between each waypoint

That is, the time taken to get from one waypoint to the next. If we think of waypoints as splits, this is essentially a `timeInSplit` value. If we know this information, we can work out how much time each competitor made, or lost, relative to every other competitor in the same class, going between each waypoint.

This means we may be able to work out which parts of the stage a particular competitor was pushing on, or had difficulties on.

There's an issue with the following: if there is missing data at a waypoint, then the `nan` causes issues with the calculations. One fix might be to try to drop a column if there's lots of missing data in it, which is the approach used in `get_annotated_timing_data()`; another might be to try to fill just the occasional `nan` across from the previous stage; this would then give a 0 time from one stage to another which we might be able to catch as some sort of exception?

In `_get_time_between_waypoints()` we go even more defensive and drop NA time rows from waypoints.

In [55]:
def _get_time_between_waypoints(timing_data_long):
    ''' Find time taken to go from one waypoint to the next for each vehicle. '''
    
    timing_data_long.dropna(subset=['TimeInS'], inplace=True)
    #The timeInSplit is the time between waypoints.
    #So find the diff between each consecutive waypoint time for each Crew
    timing_data_long['timeInSplit'] = timing_data_long[['Crew','Time']].groupby('Crew').diff()

    #Because we're using a diff(), the first row is set to NaN - there's nothing to diff to
    #So use the time at the first split as the time from the start to the first waypoint.
    timing_data_long.loc[timing_data_long.groupby('Crew')['timeInSplit'].head(1).index, 'timeInSplit'] = timing_data_long.loc[timing_data_long.groupby('Crew')['timeInSplit'].head(1).index,'Time']

    #To finesse diff calculations on NaT, set diff with day!=0 to NaT
    #This catches things where we get spurious times calculated as diff times against NaTs
    timing_data_long.loc[timing_data_long['Time'].isna(),'timeInSplit'] = pd.NaT
    timing_data_long.loc[timing_data_long['timeInSplit'].dt.days!=0,'timeInSplit'] = pd.NaT

    #If there's been a reset, we can fill across
    timing_data_long[['Time','timeInSplit']] = timing_data_long[['Time','timeInSplit']].fillna(method='ffill',axis=1)

    #Find the total seconds for each split / waypoint duration
    timing_data_long['splitS'] = timing_data_long['timeInSplit'].dt.total_seconds()
    
    return timing_data_long



def get_timing_data_long_timeInSplit(stage, vtype='car', timerank='time'):
    ''' For a stage, get the data in long form, including timeInSplit times. '''
    
    timing_data_long = get_long_annotated_timing_data(stage, vtype, timerank)[TIME]
    timing_data_long = _get_time_between_waypoints(timing_data_long)
    return timing_data_long

In [56]:
#Preview some data
timing_data_long_insplit = get_timing_data_long_timeInSplit(STAGE, VTYPE)
#timing_data_long_insplit[timing_data_long_insplit['Brand']=='PEUGEOT'].head()
timing_data_long_insplit.head()

,Pos,Bib,Road Position,Refuel,Crew,Brand,Waypoint,Time,TimeInS,timeInSplit,splitS
83,1.0,319,19.0,False,V. ZALA S. JURGELENAS AGRORODEO,AGRORODEO,01_km47,00:36:18,2178.0,00:36:18,2178.0
84,2.0,302,3.0,False,S. PETERHANSEL P. FIUZA BAHRAIN JCW X-RAID TEAM,BAHRAIN JCW X-RAID TEAM,01_km47,00:35:30,2130.0,00:35:30,2130.0
85,3.0,305,6.0,False,C. SAINZ L. CRUZ BAHRAIN JCW X-RAID TEAM,BAHRAIN JCW X-RAID TEAM,01_km47,00:34:43,2083.0,00:34:43,2083.0
86,4.0,300,1.0,False,N. AL-ATTIYAH M. BAUMEL TOYOTA GAZOO RACING,TOYOTA GAZOO RACING,01_km47,00:34:00,2040.0,00:34:00,2040.0
87,5.0,307,8.0,False,B. TEN BRINKE T. COLSOUL TOYOTA GAZOO RACING,TOYOTA GAZOO RACING,01_km47,00:36:01,2161.0,00:36:01,2161.0


In [57]:
timing_data_long = get_long_annotated_timing_data(STAGE, VTYPE)[TIME]
timing_data_long
#timing_data_long[timing_data_long['Brand']=='PEUGEOT'].head()
timing_data_long.head()

,Pos,Bib,Road Position,Refuel,Crew,Brand,Waypoint,Time,TimeInS
83,1.0,319,19.0,False,V. ZALA S. JURGELENAS AGRORODEO,AGRORODEO,01_km47,00:36:18,2178.0
84,2.0,302,3.0,False,S. PETERHANSEL P. FIUZA BAHRAIN JCW X-RAID TEAM,BAHRAIN JCW X-RAID TEAM,01_km47,00:35:30,2130.0
85,3.0,305,6.0,False,C. SAINZ L. CRUZ BAHRAIN JCW X-RAID TEAM,BAHRAIN JCW X-RAID TEAM,01_km47,00:34:43,2083.0
86,4.0,300,1.0,False,N. AL-ATTIYAH M. BAUMEL TOYOTA GAZOO RACING,TOYOTA GAZOO RACING,01_km47,00:34:00,2040.0
87,5.0,307,8.0,False,B. TEN BRINKE T. COLSOUL TOYOTA GAZOO RACING,TOYOTA GAZOO RACING,01_km47,00:36:01,2161.0


In [58]:
timing_data_long['Waypoint'].unique()

array(['01_km47', '02_km86', '04_km158', '05_km208', '06_km254',
       '07_km294', '08_ass'], dtype=object)

## Saving the Data to a Database

The data can be saved to a database directly in an unnormalised form, or we can tidy it up a bit and save it in a higher normal form.

The table structure is far from best practice - it's pragmatic and in first instance intended simply to be useful...

In [59]:
#!pip3 install sqlite-utils
from sqlite_utils import Database

In [60]:
def cleardbtable(conn, table):
    ''' Clear the table whilst retaining the table definition '''
    c = conn.cursor()
    c.execute('DELETE FROM "{}"'.format(table))
    
def dbfy(conn, df, table, if_exists='append', index=False, clear=False, **kwargs):
    ''' Save a dataframe as a SQLite table.
        Clearing or replacing a table will first empty the table of entries but retain the structure. '''
    if if_exists=='replace':
        clear=True
        if_exists='append'
    if clear: cleardbtable(conn, table)
    df.to_sql(table,conn,if_exists=if_exists,index=index)

In [61]:
#dbname='dakar_test_sql.sqlite'
#!rm $dbname

In [62]:
import sqlite3

In [63]:
dbname='dakar_sql-X.sqlite'
!rm $dbname

conn = sqlite3.connect(dbname)

c = conn.cursor()

setup_sql= 'dakar.sql'
with open(setup_sql,'r') as f:
    txt = f.read()
    c.executescript(txt)
    
db = Database(conn)

In [64]:
q="SELECT name FROM sqlite_master WHERE type = 'table';"
pd.read_sql(q, conn)

,name
0,teams
1,crew
2,vehicles
3,stagestats
4,ranking
5,waypoints


In [65]:
tmp = teamnames.reset_index()
tmp['Year'] = YEAR
tmp.rename(columns={'value':'Team'}, inplace=True)
dbfy(conn, tmp, "teams")

In [66]:
q="SELECT * FROM teams LIMIT 3;"
pd.read_sql(q, conn)

,Year,Bib,Team
0,2020,1,RED BULL KTM FACTORY TEAM
1,2020,2,RED BULL KTM FACTORY TEAM
2,2020,3,RED BULL KTM FACTORY TEAM


In [67]:
tmp = personnel[['Bib','Num','Name','Country']]
tmp['Year'] = YEAR
dbfy(conn, tmp, "crew")

In [68]:
q="SELECT * FROM crew LIMIT 3;"
pd.read_sql(q, conn)

,Year,Bib,Num,Name,Country
0,2020,1,0,T. PRICE,Australia
1,2020,2,0,M. WALKNER,Austria
2,2020,3,0,S. SUNDERLAND,United Kingdom


In [69]:
import os

def init_db(dbname='file::memory:', setup_sql='dakar.sql', clean=True):
    
    if clean and os.path.exists(dbname):
        #!rm $dbname
        os.remove(dbname)

    conn = sqlite3.connect(dbname)

    c = conn.cursor()

    with open(setup_sql,'r') as f:
        txt = f.read()
        c.executescript(txt)

    db = Database(conn)
    
    tmp = teamnames.reset_index()
    tmp['Year'] = YEAR
    tmp.rename(columns={'value':'Team'}, inplace=True)
    dbfy(conn, tmp, "teams")
    
    tmp = personnel[['Bib','Num','Name','Country']]
    tmp['Year'] = YEAR
    dbfy(conn, tmp, "crew")
    
    return conn, db

In [70]:
stage=1

tmp = get_stage_stats(STAGE).set_index('Special').T.reset_index()
tmp.rename(columns={'Leader at latest WP':'LeaderLatestWP', 'index':'Vehicle', "Latest WP":'LatestWP',
                    'At start':'AtStart', 'Nb at latest WP':'NumLatestWP' }, inplace=True)
tmp[['BibLatestWP', 'NameLatestWP']] = tmp['LeaderLatestWP'].str.extract(r'([^ ]*) (.*)',expand=True)
tmp['BibLatestWP'] = tmp['BibLatestWP'].astype(int)

tmp['Stage'] = stage
tmp['Year'] = YEAR

tmp['StageDist'] = tmp['Special'].str.extract(r'(?P<StageDist>.*)[pkm]{2}').astype(int)

stagedists = tmp[['Stage', 'Vehicle', 'StageDist']]
tmp

Special,Vehicle,Start,Liaison,Special,Number of participants,AtStart,Left,Arrived,LatestWP,LeaderLatestWP,NumLatestWP,BibLatestWP,NameLatestWP,Stage,Year,StageDist
0,Moto,07:20,433km,319km,Number of participants,144,144,140,ass,001 PRICE,140,1,PRICE,1,2020,319
1,Quad,08:34,433km,319km,Number of participants,23,23,22,ass,250 CASALE,22,250,CASALE,1,2020,319
2,Car,09:20,433km,319km,Number of participants,83,82,81,ass,319 ZALA,81,319,ZALA,1,2020,319
3,SSV,10:49,433km,319km,Number of participants,46,46,45,ass,419 DOMZALA,45,419,DOMZALA,1,2020,319
4,Truck,11:50,433km,319km,Number of participants,46,41,40,ass,516 SHIBALOV,40,516,SHIBALOV,1,2020,319


In [71]:
stagedists

Special,Stage,Vehicle,StageDist
0,1,Moto,319
1,1,Quad,319
2,1,Car,319
3,1,SSV,319
4,1,Truck,319


In [72]:
### wtf is going on w/ waypoints #1?

v=VTYPE

tmp = get_long_annotated_timing_data(stage,vtype=v, timerank='time')[TIME]
tmp['Year'] = YEAR
tmp['Stage'] = stage

tmp.rename(columns={'Road Position':'RoadPos'}, inplace=True)
#try:
#    t_stagemeta.upsert_all(tmp[['Year', 'Stage', 'Bib','RoadPos','Refuel']].to_dict(orient='records'))
#except:
#    t_stagemeta.insert_all(tmp[['Year', 'Stage', 'Bib','RoadPos','Refuel']].to_dict(orient='records'))
print(tmp['Waypoint'].unique())
tmp.head()

['01_km47' '02_km86' '04_km158' '05_km208' '06_km254' '07_km294' '08_ass']


,Pos,Bib,RoadPos,Refuel,Crew,Brand,Waypoint,Time,TimeInS,Year,Stage
83,1.0,319,19.0,False,V. ZALA S. JURGELENAS AGRORODEO,AGRORODEO,01_km47,00:36:18,2178.0,2020,1
84,2.0,302,3.0,False,S. PETERHANSEL P. FIUZA BAHRAIN JCW X-RAID TEAM,BAHRAIN JCW X-RAID TEAM,01_km47,00:35:30,2130.0,2020,1
85,3.0,305,6.0,False,C. SAINZ L. CRUZ BAHRAIN JCW X-RAID TEAM,BAHRAIN JCW X-RAID TEAM,01_km47,00:34:43,2083.0,2020,1
86,4.0,300,1.0,False,N. AL-ATTIYAH M. BAUMEL TOYOTA GAZOO RACING,TOYOTA GAZOO RACING,01_km47,00:34:00,2040.0,2020,1
87,5.0,307,8.0,False,B. TEN BRINKE T. COLSOUL TOYOTA GAZOO RACING,TOYOTA GAZOO RACING,01_km47,00:36:01,2161.0,2020,1


In [73]:
### wtf is going on w/ waypoints #2?
tmp2 = get_long_annotated_timing_data(stage,vtype=v, timerank='gap')[TIME]
tmp = pd.merge(tmp, tmp2[['Bib', 'Waypoint', 'Gap', 'GapInS']],
               on=['Bib', 'Waypoint'], how='left')
tmp['WaypointOrder'] = tmp['Waypoint'].str.slice(0,2).astype(int)

tmp['WaypointOrder'] = tmp['Waypoint'].str.slice(0,2).astype(int)

print(tmp['Waypoint'].unique())
tmp.head()

['01_km47' '02_km86' '04_km158' '05_km208' '06_km254' '07_km294' '08_ass']


,Pos,Bib,RoadPos,Refuel,Crew,Brand,Waypoint,Time,TimeInS,Year,Stage,Gap,GapInS,WaypointOrder
0,1.0,319,19.0,False,V. ZALA S. JURGELENAS AGRORODEO,AGRORODEO,01_km47,00:36:18,2178.0,2020,1,00:02:18,138.0,1
1,2.0,302,3.0,False,S. PETERHANSEL P. FIUZA BAHRAIN JCW X-RAID TEAM,BAHRAIN JCW X-RAID TEAM,01_km47,00:35:30,2130.0,2020,1,00:01:30,90.0,1
2,3.0,305,6.0,False,C. SAINZ L. CRUZ BAHRAIN JCW X-RAID TEAM,BAHRAIN JCW X-RAID TEAM,01_km47,00:34:43,2083.0,2020,1,00:00:43,43.0,1
3,4.0,300,1.0,False,N. AL-ATTIYAH M. BAUMEL TOYOTA GAZOO RACING,TOYOTA GAZOO RACING,01_km47,00:34:00,2040.0,2020,1,00:00:00,0.0,1
4,5.0,307,8.0,False,B. TEN BRINKE T. COLSOUL TOYOTA GAZOO RACING,TOYOTA GAZOO RACING,01_km47,00:36:01,2161.0,2020,1,00:02:01,121.0,1


In [74]:
### wtf is going on w/ waypoints #3?
tmp['VehicleType'] = v

#Add in the WaypointRank
tmp3=get_annotated_timing_data(stage,vtype=v, timerank='time', kind='full')[TIME]

tmp = pd.merge(tmp, _typ_long(tmp3, '_pos_')[['Bib','WaypointOrder', '_pos_']],
               on=['Bib','WaypointOrder'], how='left')
tmp.rename(columns={'_pos_':'WaypointRank'}, inplace=True)
print(tmp['Waypoint'].unique())
tmp.head()

['01_km47' '02_km86' '04_km158' '05_km208' '06_km254' '07_km294' '08_ass']


,Pos,Bib,RoadPos,Refuel,Crew,Brand,Waypoint,Time,TimeInS,Year,Stage,Gap,GapInS,WaypointOrder,VehicleType,WaypointRank
0,1.0,319,19.0,False,V. ZALA S. JURGELENAS AGRORODEO,AGRORODEO,01_km47,00:36:18,2178.0,2020,1,00:02:18,138.0,1,car,11.0
1,2.0,302,3.0,False,S. PETERHANSEL P. FIUZA BAHRAIN JCW X-RAID TEAM,BAHRAIN JCW X-RAID TEAM,01_km47,00:35:30,2130.0,2020,1,00:01:30,90.0,1,car,7.0
2,3.0,305,6.0,False,C. SAINZ L. CRUZ BAHRAIN JCW X-RAID TEAM,BAHRAIN JCW X-RAID TEAM,01_km47,00:34:43,2083.0,2020,1,00:00:43,43.0,1,car,4.0
3,4.0,300,1.0,False,N. AL-ATTIYAH M. BAUMEL TOYOTA GAZOO RACING,TOYOTA GAZOO RACING,01_km47,00:34:00,2040.0,2020,1,00:00:00,0.0,1,car,1.0
4,5.0,307,8.0,False,B. TEN BRINKE T. COLSOUL TOYOTA GAZOO RACING,TOYOTA GAZOO RACING,01_km47,00:36:01,2161.0,2020,1,00:02:01,121.0,1,car,8.0


In [75]:
### wtf is going on w/ waypoints #4?
## ADD IN
#Way point pos


waypoints = pd.DataFrame({'Waypoint':tmp['Waypoint'].unique().tolist()})
    
waypoints['WaypointDist'] = waypoints['Waypoint'].str.extract(r'.*_[pkm]{2}(?P<Change>.*)?')
stagedists[stagedists['Vehicle'].str.lower()==v]['StageDist'].iloc[0]
#tmp
waypoints

,Waypoint,WaypointDist
0,01_km47,47
1,02_km86,86
2,04_km158,158
3,05_km208,208
4,06_km254,254
5,07_km294,294
6,08_ass,NaN


In [76]:
### wtf is going on w/ waypoints #5?
waypoints.loc[waypoints['Waypoint'].str.contains('_ass'),'WaypointDist'] = stagedists[stagedists['Vehicle'].str.lower()==v]['StageDist'].iloc[0]
waypoints['WaypointDist'] = waypoints['WaypointDist'].astype(int)
waypoints


,Waypoint,WaypointDist
0,01_km47,47
1,02_km86,86
2,04_km158,158
3,05_km208,208
4,06_km254,254
5,07_km294,294
6,08_ass,319


In [77]:
### wtf is going on w/ waypoints #6?
tmp = pd.merge(tmp, waypoints, on=['Waypoint'], how='left')
tmp

,Pos,Bib,RoadPos,Refuel,Crew,Brand,Waypoint,Time,TimeInS,Year,Stage,Gap,GapInS,WaypointOrder,VehicleType,WaypointRank,WaypointDist
0,1.0,319,19.0,False,V. ZALA S. JURGELENAS AGRORODEO,AGRORODEO,01_km47,0 days 00:36:18,2178.0,2020,1,0 days 00:02:18,138.0,1,car,11.0,47
1,2.0,302,3.0,False,S. PETERHANSEL P. FIUZA BAHRAIN JCW X-RAID TEAM,BAHRAIN JCW X-RAID TEAM,01_km47,0 days 00:35:30,2130.0,2020,1,0 days 00:01:30,90.0,1,car,7.0,47
2,3.0,305,6.0,False,C. SAINZ L. CRUZ BAHRAIN JCW X-RAID TEAM,BAHRAIN JCW X-RAID TEAM,01_km47,0 days 00:34:43,2083.0,2020,1,0 days 00:00:43,43.0,1,car,4.0,47
3,4.0,300,1.0,False,N. AL-ATTIYAH M. BAUMEL TOYOTA GAZOO RACING,TOYOTA GAZOO RACING,01_km47,0 days 00:34:00,2040.0,2020,1,0 days 00:00:00,0.0,1,car,1.0,47
4,5.0,307,8.0,False,B. TEN BRINKE T. COLSOUL TOYOTA GAZOO RACING,TOYOTA GAZOO RACING,01_km47,0 days 00:36:01,2161.0,2020,1,0 days 00:02:01,121.0,1,car,8.0,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
576,79.0,377,72.0,False,J. FONT B. RODRIGUEZ FN SPEED TEAM,FN SPEED TEAM,08_ass,2 days 11:58:48,215928.0,2020,1,2 days 08:39:44,203984.0,8,car,NaN,319
577,80.0,348,47.0,False,B. ZAPAG JJ. SANCHEZ SOUTH RACING,SOUTH RACING,08_ass,2 days 18:00:00,237600.0,2020,1,2 days 14:40:56,225656.0,8,car,NaN,319
578,81.0,374,NaN,False,A. ALYAEESH FM. FTYH NAIZK,NAIZK,08_ass,2 days 18:00:00,237600.0,2020,1,2 days 14:40:56,225656.0,8,car,NaN,319
579,NaN,329,29.0,False,R. DUMAS A. WINOCQ RD LIMITED,RD LIMITED,08_ass,NaT,NaN,2020,1,NaT,NaN,8,car,NaN,319


In [78]:
### wtf is going on w/ waypoints #7?
#TO DO: use Gap and Time as raw
#print(tmp.columns, tmp.dtypes)
_tcols = ["Time","Gap"]
for t in _tcols:
    tmp['{}InS'.format(t)] = pd.to_timedelta( tmp[t] ).dt.total_seconds()
tmp.drop(['RoadPos', 'Refuel', 'Brand', 'Crew'], axis=1, inplace=True)
tmp.drop(['Time','Gap'], axis=1, inplace=True)

tmp = pd.merge(tmp, _typ_long(tmp3, '_raw_')[['Bib','WaypointOrder', '_raw_']],
               on=['Bib','WaypointOrder'])
tmp.rename(columns={'_raw_':'Time_raw'}, inplace=True)

tmp = pd.merge(tmp, _typ_long(get_annotated_timing_data(stage,vtype=v, timerank='gap', kind='full')[TIME], '_raw_')[['Bib','WaypointOrder', '_raw_']],
               on=['Bib','WaypointOrder'])
tmp.rename(columns={'_raw_':'Gap_raw'}, inplace=True)

tmp

,Pos,Bib,Waypoint,TimeInS,Year,Stage,GapInS,WaypointOrder,VehicleType,WaypointRank,WaypointDist,Time_raw,Gap_raw
0,1.0,319,01_km47,2178.0,2020,1,138.0,1,car,11.0,47,00:36:18,00:02:18
1,2.0,302,01_km47,2130.0,2020,1,90.0,1,car,7.0,47,00:35:30,00:01:30
2,3.0,305,01_km47,2083.0,2020,1,43.0,1,car,4.0,47,00:34:43,00:00:43
3,4.0,300,01_km47,2040.0,2020,1,0.0,1,car,1.0,47,00:34:00,00:00:00
4,5.0,307,01_km47,2161.0,2020,1,121.0,1,car,8.0,47,00:36:01,00:02:01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
576,79.0,377,08_ass,215928.0,2020,1,203984.0,8,car,NaN,319,59:58:48,56:39:44
577,80.0,348,08_ass,237600.0,2020,1,225656.0,8,car,NaN,319,66:00:00,62:40:56
578,81.0,374,08_ass,237600.0,2020,1,225656.0,8,car,NaN,319,66:00:00,62:40:56
579,NaN,329,08_ass,NaN,2020,1,NaN,8,car,NaN,319,NaN,NaN


In [79]:
get_timing_data_long_timeInSplit(stage, VTYPE)[['Bib','Waypoint', 'timeInSplit','splitS']]

,Bib,Waypoint,timeInSplit,splitS
83,319,01_km47,0 days 00:36:18,2178.0
84,302,01_km47,0 days 00:35:30,2130.0
85,305,01_km47,0 days 00:34:43,2083.0
86,300,01_km47,0 days 00:34:00,2040.0
87,307,01_km47,0 days 00:36:01,2161.0
...,...,...,...,...
657,303,08_ass,0 days 00:21:40,1300.0
658,389,08_ass,0 days 00:23:47,1427.0
659,377,08_ass,2 days 11:58:48,215928.0
660,348,08_ass,2 days 18:00:00,237600.0


In [80]:
### wtf is going on w/ waypoints #8?
tmp = pd.merge(tmp, get_timing_data_long_timeInSplit(stage, v)[['Bib','Waypoint', 'timeInSplit','splitS']],
               on=['Bib','Waypoint'])
tmp['WaypointPos'] = tmp.groupby(['Stage','Waypoint'])['splitS'].rank(ascending=True,
                                                                       method='dense')
tmp.head(3)

,Pos,Bib,Waypoint,TimeInS,Year,Stage,GapInS,WaypointOrder,VehicleType,WaypointRank,WaypointDist,Time_raw,Gap_raw,timeInSplit,splitS,WaypointPos
0,1.0,319,01_km47,2178.0,2020,1,138.0,1,car,11.0,47,00:36:18,00:02:18,00:36:18,2178.0,10.0
1,2.0,302,01_km47,2130.0,2020,1,90.0,1,car,7.0,47,00:35:30,00:01:30,00:35:30,2130.0,7.0
2,3.0,305,01_km47,2083.0,2020,1,43.0,1,car,4.0,47,00:34:43,00:00:43,00:34:43,2083.0,4.0


In [81]:
def create_waypoints_db_df(stage, v, stagedists):

    tmp = get_long_annotated_timing_data(stage,vtype=v, timerank='time')
    if not tmp:
        return pd.DataFrame()
        
    tmp = tmp[TIME]
    tmp['Year'] = YEAR
    tmp['Stage'] = stage

    tmp.rename(columns={'Road Position':'RoadPos'}, inplace=True)
    
    tmp2 = get_long_annotated_timing_data(stage,vtype=v, timerank='gap')[TIME]
    tmp = pd.merge(tmp, tmp2[['Bib', 'Waypoint', 'Gap', 'GapInS']],
                   on=['Bib', 'Waypoint'], how='left')
    tmp['WaypointOrder'] = tmp['Waypoint'].str.slice(0,2).astype(int)
    
    tmp['VehicleType'] = v

    #Add in the WaypointRank
    tmp3=get_annotated_timing_data(stage,vtype=v, timerank='time', kind='full')[TIME]

    tmp = pd.merge(tmp, _typ_long(tmp3, '_pos_')[['Bib','WaypointOrder', '_pos_']],
                   on=['Bib','WaypointOrder'], how='left')
    tmp.rename(columns={'_pos_':'WaypointRank'}, inplace=True)
    
    tmp['WaypointPos'] = tmp.groupby(['Stage','Waypoint'])['GapInS'].rank(ascending=True,
                                                                       method='dense')
    
    waypoints = pd.DataFrame({'Waypoint':tmp['Waypoint'].unique().tolist()})
    
    waypoints['WaypointDist'] = waypoints['Waypoint'].str.extract(r'.*_[pkm]{2}(?P<Change>.*)?')
    stagedists[stagedists['Vehicle'].str.lower()==v]['StageDist'].iloc[0]

    waypoints.loc[waypoints['Waypoint'].str.contains('_ass'),'WaypointDist'] = stagedists[stagedists['Vehicle'].str.lower()==v]['StageDist'].iloc[0]
    waypoints['WaypointDist'] = waypoints['WaypointDist'].astype(int)

    tmp = pd.merge(tmp, waypoints, on=['Waypoint'], how='left')

    _tcols = ["Time","Gap"]
    for t in _tcols:
        tmp['{}InS'.format(t)] = pd.to_timedelta( tmp[t] ).dt.total_seconds()
    tmp.drop(['RoadPos', 'Refuel', 'Brand', 'Crew'], axis=1, inplace=True)
    tmp.drop(['Time','Gap'], axis=1, inplace=True)

    tmp = pd.merge(tmp, _typ_long(tmp3, '_raw_')[['Bib','WaypointOrder', '_raw_']],
                   on=['Bib','WaypointOrder'])
    tmp.rename(columns={'_raw_':'Time_raw'}, inplace=True)

    tmp = pd.merge(tmp, _typ_long(get_annotated_timing_data(stage,vtype=v, timerank='gap', kind='full')[TIME], '_raw_')[['Bib','WaypointOrder', '_raw_']],
                   on=['Bib','WaypointOrder'])
    tmp.rename(columns={'_raw_':'Gap_raw'}, inplace=True)
    
    tmp = pd.merge(tmp, get_timing_data_long_timeInSplit(stage, v)[['Bib','Waypoint', 'splitS']],
               on=['Bib','Waypoint'])
    tmp['WaypointPos'] = tmp.groupby(['Stage','Waypoint'])['splitS'].rank(ascending=True,
                                                                       method='dense')
    return tmp

In [82]:
stage=1

In [83]:
create_waypoints_db_df(stage, v, stagedists)

,Pos,Bib,Waypoint,TimeInS,Year,Stage,GapInS,WaypointOrder,VehicleType,WaypointRank,WaypointPos,WaypointDist,Time_raw,Gap_raw,splitS
0,1.0,319,01_km47,2178.0,2020,1,138.0,1,car,11.0,10.0,47,00:36:18,00:02:18,2178.0
1,2.0,302,01_km47,2130.0,2020,1,90.0,1,car,7.0,7.0,47,00:35:30,00:01:30,2130.0
2,3.0,305,01_km47,2083.0,2020,1,43.0,1,car,4.0,4.0,47,00:34:43,00:00:43,2083.0
3,4.0,300,01_km47,2040.0,2020,1,0.0,1,car,1.0,1.0,47,00:34:00,00:00:00,2040.0
4,5.0,307,01_km47,2161.0,2020,1,121.0,1,car,8.0,8.0,47,00:36:01,00:02:01,2161.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546,77.0,303,08_ass,33520.0,2020,1,21576.0,8,car,NaN,54.0,319,09:18:40,05:59:36,1300.0
547,78.0,389,08_ass,35112.0,2020,1,23168.0,8,car,NaN,57.0,319,09:45:12,06:26:08,1427.0
548,79.0,377,08_ass,215928.0,2020,1,203984.0,8,car,NaN,76.0,319,59:58:48,56:39:44,215928.0
549,80.0,348,08_ass,237600.0,2020,1,225656.0,8,car,NaN,77.0,319,66:00:00,62:40:56,237600.0


In [96]:
conn, db = init_db('dakar_2020.db')

In [97]:
# 'Year', 'Bib' is a sensible common key for several things?

STAGESTATS = True

for stage in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]:
    
    tmp = get_stage_stats(STAGE).set_index('Special').T.reset_index()

    tmp.drop(['Number of participants'], axis=1, inplace=True)
    tmp.rename(columns={'Leader at latest WP':'LeaderLatestWP', 'index':'Vehicle',  "Latest WP":'LatestWP',
                'At start':'AtStart', 'Nb at latest WP':'NumLatestWP' }, inplace=True)
    tmp[['BibLatestWP', 'NameLatestWP']] = tmp['LeaderLatestWP'].str.extract(r'([^ ]*) (.*)',expand=True)
    tmp['BibLatestWP'] = tmp['BibLatestWP'].astype(int)
    tmp['Stage'] = stage
    tmp['Year'] = YEAR
    tmp['StageDist'] = tmp['Special'].str.extract(r'(?P<StageDist>.*)[pkm]{2}').astype(int)
    
    if STAGESTATS:
        db["stagestats"].upsert_all(tmp.to_dict(orient='records'),
                                pk=("Year","Stage","Vehicle"))

    stagedists = tmp[['Stage', 'Vehicle', 'StageDist']]
    
    #Need a better flg for if stages are yet to run
    _running = True
    
    for v in VTYPE_:
        #_get_news(stage,vtype=v)
        #get_stage_stats(stage)
        print(stage,v)
        
        tmp = create_waypoints_db_df(stage, v, stagedists)
        if tmp.empty:
            _running=False
            continue
        
        db["waypoints"].upsert_all(tmp.to_dict(orient='records'),
                                   pk=("Year", "Stage", "Bib", "Waypoint"))
            
        for ranking in RANKING_:  
            tmp = get_ranking_data(stage,vtype=v, timerank=ranking, kind='raw')[RANK]
            tmp['Year'] = YEAR
            
            tmp['Stage'] = stage
            tmp['Type'] = ranking
            _tcols = ["Time","Gap","Penalty"]
            for t in _tcols:
                tmp['{}InS'.format(t)] = pd.to_timedelta( tmp[t] ).dt.total_seconds()
            tmp.drop(_tcols, axis=1, inplace=True)
            
            tmp['VehicleType'] = v
            db["ranking"].upsert_all(tmp.to_dict(orient='records'),
                                     pk=("Year", "Stage", "Type", "Bib"))
        
        #Although derived from either ranking table, only need to collect this once
        tmp = tmp[['Year','Bib','VehicleType', 'Brand']]

        #db["vehicles"].upsert_all(tmp.to_dict(orient='records'), pk=("Year","Bib"))
            
    if not _running:
        print("I guess we've not got that far yet...")
        #break

1 car
1 moto
1 quad
1 ssv
1 truck
2 car
2 moto
2 quad
2 ssv
2 truck
3 car
3 moto
3 quad
3 ssv
3 truck
4 car
4 moto
4 quad
4 ssv
4 truck
5 car
5 moto
5 quad
5 ssv
5 truck
6 car
6 moto
6 quad
6 ssv
6 truck
7 car
7 moto
7 quad
7 ssv
7 truck
8 car
8 moto
8 quad
8 ssv
8 truck
I guess we've not got that far yet...
9 car
9 moto
9 quad
9 ssv
9 truck
10 car
10 moto
10 quad
10 ssv
10 truck
11 car
11 moto
11 quad
11 ssv
11 truck
12 car
12 moto
12 quad
12 ssv
12 truck


## Report Generation

NOTE: The intention is to migrate code into a `.py` file generated from this notebook using Jupytext, thence called from the `Dakar Stage` notebook. (There is also code in `Dakar Overall` that probably should move to, or may already be found in, the `.py` file. 

In [ ]:
q="SELECT * FROM stagestats;"
pd.read_sql(q, conn)

In [ ]:
q="SELECT * FROM vehicles;"
pd.read_sql(q, conn)

In [ ]:
q="SELECT * FROM waypoints LIMIT 20;"
pd.read_sql(q, conn)

In [ ]:
q="SELECT DISTINCT VehicleType FROM vehicles;"
pd.read_sql(q, conn)

In [96]:
q="SELECT * FROM ranking LIMIT 3;"
pd.read_sql(q, conn)

,Year,Stage,Type,Pos,Bib,VehicleType,Crew,Brand,Time_raw,TimeInS,Gap_raw,GapInS,Penalty_raw,PenaltyInS
0,2020,1,stage,1,319,car,V. ZALA S. JURGELENAS AGRORODEO,MINI,03:19:04,11944,0:00:00,0,00:00:00,0
1,2020,1,stage,2,302,car,S. PETERHANSEL P. FIUZA BAHRAIN JCW X-RAID TEAM,MINI,03:21:18,12078,0:02:14,134,00:00:00,0
2,2020,1,stage,3,305,car,C. SAINZ L. CRUZ BAHRAIN JCW X-RAID TEAM,MINI,03:21:54,12114,0:02:50,170,00:00:00,0


In [97]:
q="SELECT * FROM ranking WHERE Type='general' AND Stage=1 AND Pos=4 LIMIT 5;"
pd.read_sql(q, conn)

,Year,Stage,Type,Pos,Bib,VehicleType,Crew,Brand,Time_raw,TimeInS,Gap_raw,GapInS,Penalty_raw,PenaltyInS
0,2020,1,general,4,300,car,N. AL-ATTIYAH M. BAUMEL TOYOTA GAZOO RACING,TOYOTA,03:24:37,12277.0,0:05:33,333.0,00:00:00,0
1,2020,1,general,4,7,moto,K. BENAVIDES MONSTER ENERGY HONDA TEAM 2020,HONDA,03:24:04,12244.0,0:02:31,151.0,00:00:00,0
2,2020,1,general,4,255,quad,M. ANDUJAR 7240 TEAM,YAMAHA,04:36:18,16578.0,0:18:41,1121.0,00:07:00,420
3,2020,1,general,4,400,ssv,F. LOPEZ CONTARDO JP. LATRACH VINAGRE SOUTH RA...,CAN - AM,04:07:45,14865.0,0:06:47,407.0,00:00:00,0
4,2020,1,general,4,501,truck,D. SOTNIKOV R. AKHMADEEV I. AKHMETZIANOV KAMAZ...,KAMAZ,03:44:42,13482.0,0:04:07,247.0,00:00:00,0


In [98]:
q="SELECT DISTINCT Brand FROM vehicles WHERE VehicleType='ssv' ;"
pd.read_sql(q, conn)

,Brand


In [99]:
q="SELECT w.* FROM waypoints w JOIN vehicles v ON w.Bib=v.Bib WHERE Stage=1 AND w.VehicleType='car' AND Pos<=10"
tmpq = pd.read_sql(q, conn)
tmpq.pivot(index='Bib',columns='Waypoint',values='TimeInS')

Waypoint
Bib


In [100]:
#Example - driver overall ranks by stage in in top10 overall at end of stage
q="SELECT * FROM ranking WHERE VehicleType='car' AND Type='general' AND Pos<=10"
tmpq = pd.read_sql(q, conn)
tmpq.pivot(index='Bib',columns='Stage',values='Pos')

Stage,1,2,3,4,5
Bib,,,,,
300,4.0,3.0,2.0,2.0,2.0
302,2.0,8.0,5.0,3.0,3.0
304,NaN,6.0,7.0,7.0,7.0
305,3.0,2.0,1.0,1.0,1.0
306,NaN,NaN,9.0,9.0,NaN
307,5.0,10.0,8.0,8.0,8.0
308,NaN,7.0,NaN,NaN,NaN
309,9.0,9.0,4.0,4.0,4.0
311,6.0,1.0,3.0,5.0,5.0


In [101]:
tmpq

,Year,Stage,Type,Pos,Bib,VehicleType,Crew,Brand,Time_raw,TimeInS,Gap_raw,GapInS,Penalty_raw,PenaltyInS
0,2020,1,general,1,319,car,V. ZALA S. JURGELENAS AGRORODEO,MINI,03:19:04,11944.0,0:00:00,0.0,00:00:00,0.0
1,2020,1,general,2,302,car,S. PETERHANSEL P. FIUZA BAHRAIN JCW X-RAID TEAM,MINI,03:21:18,12078.0,0:02:14,134.0,00:00:00,0.0
2,2020,1,general,3,305,car,C. SAINZ L. CRUZ BAHRAIN JCW X-RAID TEAM,MINI,03:21:54,12114.0,0:02:50,170.0,00:00:00,0.0
3,2020,1,general,4,300,car,N. AL-ATTIYAH M. BAUMEL TOYOTA GAZOO RACING,TOYOTA,03:24:37,12277.0,0:05:33,333.0,00:00:00,0.0
4,2020,1,general,5,307,car,B. TEN BRINKE T. COLSOUL TOYOTA GAZOO RACING,TOYOTA,03:25:34,12334.0,0:06:30,390.0,00:00:00,0.0
5,2020,1,general,6,311,car,O. TERRANOVA B. GRAUE X-RAID MINI JCW TEAM,MINI,03:26:19,12379.0,0:07:15,435.0,00:00:00,0.0
6,2020,1,general,7,315,car,M. SERRADORI F. LURQUIN SRT RACING,CENTURY,03:27:59,12479.0,0:08:55,535.0,00:02:00,120.0
7,2020,1,general,8,317,car,V. VASILYEV V. YEVTYEKHOV X-RAID G-ENERGY,MINI,03:32:29,12749.0,0:13:25,805.0,00:00:00,0.0
8,2020,1,general,9,309,car,Y. AL RAJHI K. ZHILTSOV OVERDRIVE TOYOTA,TOYOTA,03:32:50,12770.0,0:13:46,826.0,00:02:00,120.0
9,2020,1,general,10,314,car,E. VAN LOON S. DELAUNAY OVERDRIVE TOYOTA,TOYOTA,03:33:02,12782.0,0:13:58,838.0,00:00:00,0.0


In [102]:
#Driver top 5 finishes by stage
q="SELECT r.*, c.Name FROM ranking r LEFT JOIN crew c ON c.Bib=r.Bib WHERE Num=0 AND VehicleType='car' AND Type='stage' AND Pos<=5"

tmpq = pd.read_sql(q, conn)
tmpq.pivot(index='Stage',columns='Pos',values='Name')

Pos,1,2,3,4,5
Stage,,,,,
1,V. ZALA,S. PETERHANSEL,C. SAINZ,N. AL-ATTIYAH,B. TEN BRINKE
2,G. DE VILLIERS,O. TERRANOVA,SK. AL QASSIMI,M. SERRADORI,N. AL-ATTIYAH
3,C. SAINZ,J. PRZYGONSKI,Y. SEAIDAN,F. ALONSO,N. AL-ATTIYAH
4,S. PETERHANSEL,N. AL-ATTIYAH,C. SAINZ,Y. AL RAJHI,M. SERRADORI
5,C. SAINZ,N. AL-ATTIYAH,S. PETERHANSEL,Y. AL RAJHI,O. TERRANOVA


# Reset Base Dataframes

That is, reset for the particular data config we defined at the top of the notebook.

## Rebase relative to driver

Rebasing means finding deltas relative to a specified driver. It lets us see the deltas a particular driver has to each other driver.

In [103]:
def rebaseTimes(times, bib=None, col=None):
    if bib is None or col is None: return times
    return times[col] - times[times['Bib']==bib][col].iloc[0]

def rebaseWaypointTimes(times, bib=None, col='splitS'):
    ''' Rebase times relative to a particular competitor. '''
    
    if bib is None: return times
    bib = int(bib)
    rebase = times[times['Bib']==bib][['Waypoint',col]].set_index('Waypoint').to_dict(orient='dict')[col]
    times['rebased']=times[col]-times['Waypoint'].map(rebase)
    return times




## Rebase Overall Waypoint Times and Time In Split Times

That is, rebase the overall time in stage at each waypoint relative to a specified driver.

In [104]:
REBASER = 305
#loeb 306, AL-ATTIYAH 301, peterhansel 304
#aravind prabhakar 48 coronel 347

In [105]:
timing_data_long_min = rebaseWaypointTimes( timing_data_long , REBASER, 'TimeInS')
-timing_data_long_min.reset_index().pivot('Bib','Waypoint','rebased').head(15)

Waypoint,01_km47,02_km86,04_km158,05_km208,06_km254,07_km294,08_ass
Bib,,,,,,,
300,43.0,72.0,656.0,601.0,173.0,-83.0,-163.0
301,-272.0,-398.0,-314.0,-632.0,-923.0,-1405.0,-1522.0
302,-47.0,-41.0,586.0,225.0,186.0,97.0,36.0
303,-13.0,-20.0,-19846.0,-20019.0,-20153.0,-20813.0,-21406.0
304,-139.0,-232.0,9.0,-211.0,-748.0,-1094.0,-1226.0
305,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
306,-95.0,-187.0,184.0,-51.0,-423.0,-650.0,-767.0
307,-78.0,-155.0,333.0,215.0,-38.0,-157.0,-220.0
308,17.0,-26.0,480.0,360.0,339.0,-862.0,-915.0


In [106]:
rebaseWaypointTimes(timing_data_long_insplit,REBASER).head(10)

,Pos,Bib,Road Position,Refuel,Crew,Brand,Waypoint,Time,TimeInS,timeInSplit,splitS,rebased
83,1.0,319,19.0,False,V. ZALA S. JURGELENAS AGRORODEO,AGRORODEO,01_km47,00:36:18,2178.0,00:36:18,2178.0,95.0
84,2.0,302,3.0,False,S. PETERHANSEL P. FIUZA BAHRAIN JCW X-RAID TEAM,BAHRAIN JCW X-RAID TEAM,01_km47,00:35:30,2130.0,00:35:30,2130.0,47.0
85,3.0,305,6.0,False,C. SAINZ L. CRUZ BAHRAIN JCW X-RAID TEAM,BAHRAIN JCW X-RAID TEAM,01_km47,00:34:43,2083.0,00:34:43,2083.0,0.0
86,4.0,300,1.0,False,N. AL-ATTIYAH M. BAUMEL TOYOTA GAZOO RACING,TOYOTA GAZOO RACING,01_km47,00:34:00,2040.0,00:34:00,2040.0,-43.0
87,5.0,307,8.0,False,B. TEN BRINKE T. COLSOUL TOYOTA GAZOO RACING,TOYOTA GAZOO RACING,01_km47,00:36:01,2161.0,00:36:01,2161.0,78.0
88,6.0,311,12.0,False,O. TERRANOVA B. GRAUE X-RAID MINI JCW TEAM,X-RAID MINI JCW TEAM,01_km47,00:34:40,2080.0,00:34:40,2080.0,-3.0
89,7.0,315,15.0,False,M. SERRADORI F. LURQUIN SRT RACING,SRT RACING,01_km47,00:34:55,2095.0,00:34:55,2095.0,12.0
90,8.0,317,17.0,False,V. VASILYEV V. YEVTYEKHOV X-RAID G-ENERGY,X-RAID G-ENERGY,01_km47,00:38:23,2303.0,00:38:23,2303.0,220.0
91,9.0,309,10.0,False,Y. AL RAJHI K. ZHILTSOV OVERDRIVE TOYOTA,OVERDRIVE TOYOTA,01_km47,00:44:47,2687.0,00:44:47,2687.0,604.0
92,10.0,314,14.0,False,E. VAN LOON S. DELAUNAY OVERDRIVE TOYOTA,OVERDRIVE TOYOTA,01_km47,00:40:42,2442.0,00:40:42,2442.0,359.0


In [107]:
def pivotRebasedSplits(rebasedSplits):
    ''' For each driver row, find the split. '''
    
    #If there are no splits...
    if rebasedSplits.empty:
        return pd.DataFrame(columns=['Bib']).set_index('Bib')
    
    rbp=-rebasedSplits.pivot('Bib','Waypoint','rebased')
    rbp.columns=['D{}'.format(c) for c in rbp.columns]
    #The columns seem to be sorted? Need to sort in actual split order
    rbp.sort_values(rbp.columns[-1],ascending =True)
    return rbp

In [108]:
tmp = pivotRebasedSplits(rebaseWaypointTimes(timing_data_long_insplit,REBASER))
tmp.head(3)

,D01_km47,D02_km86,D04_km158,D05_km208,D06_km254,D07_km294,D08_ass
Bib,,,,,,,
300,43.0,29.0,584.0,-55.0,-428.0,-256.0,-80.0
301,-272.0,-126.0,84.0,-318.0,-291.0,-482.0,-117.0
302,-47.0,6.0,627.0,-361.0,-39.0,-89.0,-61.0


In [109]:
#top10 = driver_data[(driver_data['Pos']>=45) & (driver_data['Pos']<=65)]
top10 = driver_data[(driver_data['Pos']<=20)]
top10.set_index('Bib', inplace=True)
top10

,Pos,Road Position,Refuel,Crew,Brand
Bib,,,,,
319,1.0,19.0,False,V. ZALA S. JURGELENAS AGRORODEO,AGRORODEO
302,2.0,3.0,False,S. PETERHANSEL P. FIUZA BAHRAIN JCW X-RAID TEAM,BAHRAIN JCW X-RAID TEAM
305,3.0,6.0,False,C. SAINZ L. CRUZ BAHRAIN JCW X-RAID TEAM,BAHRAIN JCW X-RAID TEAM
300,4.0,1.0,False,N. AL-ATTIYAH M. BAUMEL TOYOTA GAZOO RACING,TOYOTA GAZOO RACING
307,5.0,8.0,False,B. TEN BRINKE T. COLSOUL TOYOTA GAZOO RACING,TOYOTA GAZOO RACING
311,6.0,12.0,False,O. TERRANOVA B. GRAUE X-RAID MINI JCW TEAM,X-RAID MINI JCW TEAM
315,7.0,15.0,False,M. SERRADORI F. LURQUIN SRT RACING,SRT RACING
317,8.0,17.0,False,V. VASILYEV V. YEVTYEKHOV X-RAID G-ENERGY,X-RAID G-ENERGY
309,9.0,10.0,False,Y. AL RAJHI K. ZHILTSOV OVERDRIVE TOYOTA,OVERDRIVE TOYOTA


## Display Stage Tables

A lot of this work is copied directly from my WRC stage tables notebooks, so there is less explanation here about what the pieces are and how they work together.

In [110]:
#UPDATE THIS FROM WRC NOTEBOOKS TO HANDLE EXCEPTIONS, DATATYPES ETC
from IPython.core.display import HTML
import seaborn as sns

from numpy import NaN
from math import nan

def bg_color(s):
    ''' Set background colour sensitive to time gained or lost.
    '''
    attrs=[]
    for _s in s:        
        if _s < 0:
            attr = 'background-color: green; color: white'
        elif _s > 0: 
            attr = 'background-color: red; color: white'
        else:
            attr = ''
        attrs.append(attr)
    return attrs

In [111]:
#https://pandas.pydata.org/pandas-docs/stable/style.html
def color_negative(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    if isinstance(val, str) or pd.isnull(val): return ''
    
    
    val = val.total_seconds() if isinstance(val,pd._libs.tslibs.timedeltas.Timedelta) else val
    
    if val and (isinstance(val,int) or isinstance(val,float)):
        color = 'green' if val < 0 else 'red' if val > 0  else 'black'
    else:
        color='white'
    return 'color: %s' % color

In [112]:
def cleanDriverSplitReportBaseDataframe(rb2, ss):
    ''' Tidy up the driver split report dataframe, replacing 0 values with NaNs that can be hidden.
        Check column names and data types. '''
    
    #TO DO: set proper colnames
    if rb2.empty: return rb2
    
    rb2=rb2.replace(0,NaN)
    #rb2=rb2.fillna('') #This casts columns containing NA to object type which means we can't use nan processing
    
    #rb2['Road Position']=rb2['Road Position'].astype(float)
    return rb2

def __styleDriverSplitReportBaseDataframe(rb2, ss):
    ''' Test if basic dataframe styling.
        DEPRECATED. '''
    
    cm=sns.light_palette((210, 90, 60), input="husl",as_cmap=True)

    s=(rb2.fillna('').style
        .applymap(color_negative, subset=[c for c in rb2.columns if c not in ['Pos','Road Position','Crew','Brand'] ])
        .background_gradient(cmap=cm, subset=['Road Position'])
      )
    #data.style.applymap(highlight_cols, subset=pd.IndexSlice[:, ['B', 'C']])

    s.set_caption("{}: running times and deltas between each checkpoint.".format(ss))
    return s

In [ ]:
rb2c = pivotRebasedSplits(rebaseWaypointTimes(timing_data_long_insplit,REBASER))
rb2c = cleanDriverSplitReportBaseDataframe(rb2c, STAGE)
rb2c

In [ ]:
#rb2cTop10 = rb2c[rb2c.index.isin(top10['Bib'])]
rb2cTop10 = pd.merge(top10, rb2c, how='left', left_index=True,right_index=True)
rb2cTop10.head(2)

In [ ]:
#Need processing on the below - also change column order
newColOrder = rb2cTop10.columns[1:].tolist()+[rb2cTop10.columns[0]]
rb2cTop10=rb2cTop10[newColOrder]
rb2cTop10 = pd.merge(rb2cTop10, -timing_data_long_min.reset_index().pivot('Bib','Waypoint','rebased'), how='left', left_index=True,right_index=True)


#Cast s to timedelta

#for c in [c for c in rb2cTop10.columns if c.startswith('0')]:
#    rb2cTop10[c]=rb2cTop10[c].apply(lambda x: pd.to_timedelta('{}00:00:{}'.format('-' if x<0 else '', '0' if pd.isnull(x) else abs(x))))

#Rename last column
rb2cTop10.rename(columns={rb2cTop10.columns[-1]:'Stage Overall'}, inplace=True)


#Drop refuel column
rb2cTop10.drop('Refuel', axis=1, inplace=True)

rb2cTop10.head(3)

In [ ]:
s = __styleDriverSplitReportBaseDataframe(rb2cTop10, 'Stage {}'.format(STAGE))
html=s.render()
display(HTML(html))

In [ ]:
#Changes from WRC
#applymap(color_negative, - change column identification
#final apply - add 'Stage Overall' to list
#.background_gradient Add Pos
def moreStyleDriverSplitReportBaseDataframe(rb2,ss, caption=None):
    ''' Style the driver split report dataframe. '''
    
    if rb2.empty: return ''
        
    def _subsetter(cols, items):
        ''' Generate a subset of valid columns from a list. '''
        return [c for c in cols if c in items]
    
    
    #https://community.modeanalytics.com/gallery/python_dataframe_styling/
    # Set CSS properties for th elements in dataframe
    th_props = [
      ('font-size', '11px'),
      ('text-align', 'center'),
      ('font-weight', 'bold'),
      ('color', '#6d6d6d'),
      ('background-color', '#f7f7f9')
      ]

    # Set CSS properties for td elements in dataframe
    td_props = [
      ('font-size', '11px'),
      ]

    # Set table styles
    styles = [
      dict(selector="th", props=th_props),
      dict(selector="td", props=td_props)
      ]
    
    #Define colour palettes
    #cmg = sns.light_palette("green", as_cmap=True)
    #The blue palette helps us scale the Road Position column
    # This may help us to help identify any obvious road position effect when sorting stage times by stage rank
    cm=sns.light_palette((210, 90, 60), input="husl",as_cmap=True)
    s2=(rb2.style
        .background_gradient(cmap=cm, subset=_subsetter(rb2.columns, ['Road Position', 'Pos','Overall Position', 'Previous Overall Position']))
        .applymap(color_negative,
                  subset=[c for c in rb2.columns if rb2[c].dtype==float and (not c.startswith('D') and c not in ['Overall Position','Overall Gap','Road Position', 'Pos'])])
        .highlight_min(subset=_subsetter(rb2.columns, ['Overall Position','Previous Overall Position']), color='lightgrey')
        .highlight_max(subset=_subsetter(rb2.columns, ['Overall Time', 'Overall Gap']), color='lightgrey')
        .highlight_max(subset=_subsetter(rb2.columns, ['Previous']), color='lightgrey')
        .apply(bg_color,subset=_subsetter(rb2.columns, ['{} Overall'.format(ss), 'Overall Time','Overall Gap', 'Previous', 'Stage Overall']))
        .bar(subset=[c for c in rb2.columns if str(c).startswith('D')], align='zero', color=[ '#5fba7d','#d65f5f'])
        .set_table_styles(styles)
        #.format({'total_amt_usd_pct_diff': "{:.2%}"})
       )
    
    if caption is not None:
        s2.set_caption(caption)

    #nan issue: https://github.com/pandas-dev/pandas/issues/21527
    return s2.render().replace('nan','')

In [ ]:
## Can we style the type in the red/green thing by using timedelta and formatter
# eg https://docs.python.org/3.4/library/datetime.html?highlight=weekday#datetime.date.__format__
#and https://stackoverflow.com/a/46370761/454773

#Maybe also add a sparkline? Need to set a common y axis on all charts?

In [ ]:
s2 = moreStyleDriverSplitReportBaseDataframe(rb2cTop10, STAGE)
display(HTML(s2))

# Sparklines

Sparklines are small, "in-cell" charts that can be used to summarise trend behaviour across multiple columns in a single dtaa table row.

If we visualise the time gap relative to each other driver across checkpoints we can gain a better idea of how the overall stage gap is evolving.

A basic sparkline simply indicates trends in terms of gradient change, although we can annotate it to show whether the final value represents a positive or negative gain overall:

In [ ]:
#https://github.com/iiSeymour/sparkline-nb/blob/master/sparkline-nb.ipynb
import matplotlib.pyplot as plt

from io import BytesIO
import urllib
import base64

def fig2inlinehtml(fig):
    figfile = BytesIO()
    fig.savefig(figfile, format='png')
    figfile.seek(0) 
    figdata_png = base64.b64encode(figfile.getvalue())
    #imgstr = '<img src="data:image/png;base64,{}" />'.format(figdata_png)
    imgstr = '<img src="data:image/png;base64,{}" />'.format(urllib.parse.quote(figdata_png))
    return imgstr


def sparkline(data, figsize=(4, 0.5), **kwags):
    """
    Returns a HTML image tag containing a base64 encoded sparkline style plot
    """
    data = list(data)
    
    fig, ax = plt.subplots(1, 1, figsize=figsize, **kwags)
    ax.plot(data)
    for k,v in ax.spines.items():
        v.set_visible(False)
    ax.set_xticks([])
    ax.set_yticks([])    

    dot = 'r.' if data[len(data) - 1] <0 else 'g.'
       
    plt.plot(len(data) - 1, data[len(data) - 1], dot)

    ax.fill_between(range(len(data)), data, len(data)*[min(data)], alpha=0.1)

    return fig2inlinehtml(fig)

Sparkline charts get messed up by NAs. If there are lots of NAs in a column, dump the column.

In [ ]:
#Get rid of NA cols - maybe do this on a theshold?
rb2cTop10.dropna(how='all',axis=1,inplace=True)

In [ ]:
#The sparkline expects a list of values, so lets create a a temporary / dummy cell
# that has a list of values from columns in the same row that we want to plot across
#For example, let's grab the gap times at each waypoint:
#  these are in columns prefixed with a digit, (for convenience, just grap on leading 0;
# if there are more than 10 waypoints, we'll need to address this...)
#Ee should really also add the Stage Overall time in too, as that's essentially the last waypoint
rb2cTop10['test']= rb2cTop10[[c for c in rb2cTop10.columns if c.startswith(('0','1','Stage Overall'))]].values.tolist()
#Swap the sign of the values
rb2cTop10['test'] = rb2cTop10['test'].apply(lambda x: [-y for y in x])
rb2cTop10.head()

In [ ]:
rb2cTop10[:3]['test'].map(sparkline)

A more informative display gain show whether the gapt time at each split / waypoint is positive or negative by using different colour fills above and below a gap of zero seconds. We can also interpolate values to remove gaps appearing in the line when the sign of the value at one way point is different to the sign of the gap at the previous waypoint.

Because the sign of the gap is indicated, we don't need to identify it with the sign colour marker at the end of the line.

In [ ]:
import scipy

def sparkline2(data, figsize=(2, 0.5), colband=(('red','green'),('red','green')),
               dot=False, typ='line', **kwargs):
    """
    Returns a HTML image tag containing a base64 encoded sparkline style plot
    """
    #data = [0 if pd.isnull(d) else d for d in data]
    
    fig, ax = plt.subplots(1, 1, figsize=figsize, **kwargs)
    
    if typ=='bar':
        color=[ colband[0][0] if c<0 else colband[0][1] for c in data  ]
        ax.bar(range(len(data)),data, color=color, width=0.8)
    else:
        #Default is line plot
        ax.plot(data, linewidth=0.0)
    
        d = scipy.zeros(len(data))

        #If we don't interpolate, we get a gap in the sections/waypoints
        #  where times change sign compared to the previous section/waypoint
        ax.fill_between(range(len(data)), data, where=data<d, interpolate=True, color=colband[0][0])
        ax.fill_between(range(len(data)), data, where=data>d, interpolate=True,  color=colband[0][1])

        if dot:
            dot = colband[1][0] if data[len(data) - 1] <0 else colband[1][1]
            plt.plot(len(data) - 1, data[len(data) - 1], dot)

    for k,v in ax.spines.items():
        v.set_visible(False)
    ax.set_xticks([])
    ax.set_yticks([])

    
    return fig2inlinehtml(fig)

In [ ]:
rb2cTop10.head(3)['test'].apply(sparkline2, typ='bar');

In [ ]:
display(HTML(rb2cTop10.head(3).style.render()))

In [ ]:
%%capture
brandcol = rb2cTop10.columns.get_loc("Brand")
rb2cTop10.insert(brandcol+1, 'Stage Gap', rb2cTop10['test'].apply(sparkline2,typ='bar'))
rb2cTop10.drop('test', axis=1, inplace=True)

In [ ]:
display(HTML(rb2cTop10.head(3).style.render()))

## Additional Sparklines

A couple more sparkline charts:

- rank across splits;
- gap to leader.

In [ ]:
# get gap to leader at each split
tmp = get_timing_data(STAGE,vtype=VTYPE, timerank='gap', kind='full')[TIME].set_index('Bib')
tmp.head()

In [ ]:
cols = [c for c in tmp.columns if c.startswith(('0','1'))]
tmp[cols]  = tmp[cols].apply(lambda x: x.dt.total_seconds())
tmp['test']= tmp[[c for c in tmp.columns if (c.startswith(('0','1')) and 'dss' not in c)]].values.tolist()
tmp['test2']= tmp[[c for c in tmp.columns if '_pos' in c]+['Pos']].values.tolist()
#Want better rank higher up
tmp['test2'] = tmp['test2'].apply(lambda x: [-y for y in x])
tmp.head()

In [ ]:
tmp.head(3)['test'].map(sparkline2);

In [ ]:
def sparklineStep(data, figsize=(2, 0.5), **kwags):
    #data = [0 if pd.isnull(d) else d for d in data]
    
    fig, ax = plt.subplots(1, 1, figsize=figsize, **kwags)
    
    plt.axhspan(-1, -3, facecolor='lightgrey', alpha=0.5)
    #ax.plot(range(len(data)), [-3]*len(data), linestyle=':', color='lightgrey')
    #ax.plot(range(len(data)), [-1]*len(data), linestyle=':', color='lightgrey')
    ax.plot(range(len(data)), [-10]*len(data), linestyle=':', color='lightgrey')
    ax.step(range(len(data)), data, where='mid')

    ax.set_ylim(top=-0.9)
        
    for k,v in ax.spines.items():
        v.set_visible(False)
    ax.set_xticks([])
    ax.set_yticks([])
    
    return fig2inlinehtml(fig)

In [ ]:
rb3c = get_timing_data(STAGE,vtype=VTYPE, timerank='gap', kind='full')[TIME].set_index('Bib')

In [ ]:
def _get_col_loc(df, col=None, pos=None, left_of=None, right_of=None):
    ''' Return column position number. '''
    if col in df.columns:
        return df.columns.get_loc(col)
    elif pos and pos <len(df.columns):
        return pos
    else:
        pos = 0
    if left_of in df.columns:
        pos = df.columns.get_loc(left_of)
    elif right_of in df.columns:
        pos = min(df.columns.get_loc(right_of)+1,len(df.columns)-1)
    return pos

def moveColumn(df, col, pos=None, left_of=None, right_of=None):
    ''' Move dataframe column adjacent to a specified column. '''
    pos = _get_col_loc(df, None, pos, left_of, right_of)
    
    data = df[col].tolist()
    df.drop(col, axis=1, inplace=True)
    df.insert(pos, col, data)

def insertColumn(df, col, data, pos=None, left_of=None, right_of=None):
    ''' Insert data in dataframe column at specified location. '''
    pos =  _get_col_loc(df, col, pos, left_of, right_of)
    #print(pos)
    df.insert(pos, col, data)

In [ ]:
rb4cTop10.insert?


In [ ]:
%%capture
rb3cTop10 = pd.merge(rb2cTop10[[]], rb3c, how='left', left_index=True,right_index=True)

cols = [c for c in rb3cTop10.columns if c.startswith(('0','1'))]
#rb2cTop10[cols]  = rb2cTop10[cols].apply(lambda x: x.dt.total_seconds())

rb3cTop10['test2']= rb3cTop10[[c for c in rb3cTop10.columns if ('_pos' in c and 'dss' not in c)]+['Pos']].values.tolist()
#Want better rank higher up
rb3cTop10['test2'] = rb3cTop10['test2'].apply(lambda x: [-y if not pd.isnull(y) else float('NaN') for y in x ])
rb3cTop10['Waypoint Rank'] = rb3cTop10['test2'].apply(sparklineStep,figsize=(0.5, 0.5))

#rb3cTop10['test']= rb3cTop10[[c for c in rb3cTop10.columns if (c.startswith(('0','1')) and 'dss' not in c)]].values.tolist()
#rb3cTop10['test'] = rb3cTop10['test'].apply(lambda x: [-y if not pd.isnull(y) else float('NaN') for y in x ])
#rb3cTop10['Gap to Leader'] =  rb3cTop10['test'].apply(sparkline2, 
#                                                      figsize=(0.5, 0.5), 
#                                                      dot=True, 
#                                                      colband=(('pink','lightgreen'),('r.','g.')))

#Get rid of NA cols - maybe do this on a threshold?
rb3cTop10.dropna(how='all',axis=1,inplace=True)

rb3cTop10['test']= rb3cTop10[[c for c in rb3cTop10.columns if (c.startswith(('0','1')) and 'dss' not in c)]].values.tolist()
rb3cTop10['test'] = rb3cTop10['test'].apply(lambda x: [-y if not pd.isnull(y) else float('NaN') for y in x ])
rb3cTop10['Gap to Leader'] =  rb3cTop10['test'].apply(sparkline2, 
                                                      figsize=(0.5, 0.5), 
                                                      dot=True,
                                                      colband=(('pink','lightgreen'),('r.','g.')))

#rb3cTop10.drop('test', axis=1, inplace=True)
#rb3cTop10.drop('test2', axis=1, inplace=True)

#Use a copy for nw, while testing
rb4cTop10 = rb2cTop10.copy()
insertColumn(rb4cTop10, 'Gap to Leader', rb3cTop10['Gap to Leader'], right_of='Pos')
insertColumn(rb4cTop10, 'Waypoint Rank', rb3cTop10['Waypoint Rank'], right_of='Brand')

#rb4cTop10 = pd.merge(rb2cTop10, rb3cTop10[['Gap to Leader','Waypoint Rank']], left_index=True,right_index=True)


In [ ]:
rb4cTop10.head(2)

## Add in Overall Position

Bring in overall position data from end of stage and end of previous stage

In [ ]:
#Overall Position, Previous
tmp = get_ranking_data(STAGE, VTYPE,timerank='general')[RANK].head()
tmp

In [ ]:
def getCurrPrevOverallRank(stage, vtype='car', rebase=None):
    curr = get_ranking_data(stage, vtype,timerank='general')[RANK]
    curr.rename(columns={'Time':'Overall Time', 'Pos':'Overall Position'}, inplace=True)
    curr['Overall Gap'] = curr['Gap'].dt.total_seconds()
    if stage>1:
        prev = get_ranking_data(stage-1, vtype,timerank='general')[RANK]
        prev.rename(columns={'Pos':'Previous Overall Position',
                             'Time':'Previous Time',
                             'Gap':'Previous Gap'}, inplace=True)
        prev['Previous'] = prev['Previous Gap'].dt.total_seconds()
    else:
        prev=pd.DataFrame({'Bib':curr['Bib'],'Previous Overall Position':NaN,
                           'Previous Time':pd.Timedelta(''),
                           'Previous Gap':pd.Timedelta(''), 
                           'Previous':pd.Timedelta('')})
        
    #if rebase we need to rebase prev['Previous'] and curr['Overall Gap']
    if rebase:
        #in rebaser, note the ploarity of gap
        prev['Previous'] = -rebaseTimes(prev, bib=rebase, col='Previous Gap')
        prev['Previous'] = prev['Previous'].dt.total_seconds()
        curr['Overall Gap'] = -rebaseTimes(curr, bib=rebase, col='Overall Gap')
    df = pd.merge(curr[['Bib','Overall Position', 'Overall Gap']],
                    prev[['Bib','Previous Overall Position','Previous Time', 'Previous']],
                    on='Bib')
    
    return df.set_index('Bib')

In [ ]:
getCurrPrevOverallRank(1, rebase=REBASER).head().dtypes

In [ ]:
getCurrPrevOverallRank(STAGE, VTYPE, rebase=REBASER).head()

In [ ]:
rb4cTop10 = pd.merge(rb4cTop10,
                     getCurrPrevOverallRank(STAGE, VTYPE, rebase=REBASER)[['Overall Position',
                                                                            'Previous Overall Position',
                                                                           'Overall Gap', 'Previous']],
                     left_index=True, right_index=True)
moveColumn(rb4cTop10, 'Previous', left_of='Crew')
moveColumn(rb4cTop10, 'Previous Overall Position', left_of='Previous')
moveColumn(rb4cTop10, 'Overall Position', right_of='Pos')
moveColumn(rb4cTop10, 'Overall Gap', left_of='Pos')
rb4cTop10.head(3)

In [ ]:
rb4cTop10.columns

## Table Image Grabber

One way of capturing the table output is to render the HTML page in a browser and then grab a screenshot of the contents of the browser window.

The `selenium` web testing framework can help us achieve this.

The code below is taken from my WRC notebooks, so there is less commentary about what the pieces are and how they fit together than there might otherwise be...

Changes to outputter - comment out set_window_size to allow browser to get full table, full table grabber

In [ ]:
import os
import time
from selenium import webdriver

#Via https://stackoverflow.com/a/52572919/454773
def setup_screenshot(driver,path):
    # Ref: https://stackoverflow.com/a/52572919/
    original_size = driver.get_window_size()
    required_width = driver.execute_script('return document.body.parentNode.scrollWidth')
    required_height = driver.execute_script('return document.body.parentNode.scrollHeight')
    driver.set_window_size(required_width, required_height)
    # driver.save_screenshot(path)  # has scrollbar
    driver.find_element_by_tag_name('body').screenshot(path)  # avoids scrollbar
    driver.set_window_size(original_size['width'], original_size['height'])
    

def getTableImage(url, fn='dummy_table', basepath='.', path='.', delay=5, height=420, width=800):
    ''' Render HTML file in browser and grab a screenshot. '''
    browser = webdriver.Chrome()
    #browser.set_window_size(width, height)
    browser.get(url)
    #Give the map tiles some time to load
    time.sleep(delay)
    imgpath='{}/{}.png'.format(path,fn)
    imgfn = '{}/{}'.format(basepath, imgpath)
    imgfile = '{}/{}'.format(os.getcwd(),imgfn)
    
    setup_screenshot(browser,imgfile)
    browser.quit()
    os.remove(imgfile.replace('.png','.html'))
    #print(imgfn)
    return imgpath


def getTablePNG(tablehtml,basepath='.', path='testpng', fnstub='testhtml'):
    ''' Save HTML table as file. '''
    if not os.path.exists(path):
        os.makedirs('{}/{}'.format(basepath, path))
    fn='{cwd}/{basepath}/{path}/{fn}.html'.format(cwd=os.getcwd(), basepath=basepath, path=path,fn=fnstub)
    tmpurl='file://{fn}'.format(fn=fn)
    with open(fn, 'w') as out:
        out.write(tablehtml)
    return getTableImage(tmpurl, fnstub, basepath, path)

We get some nonsense in charts if there are missing values.

In [ ]:
#TO DO - how about if we colour the Overall pos
# eg increasing red for further ahead, increasing green for further behind?
#or colour depending on whether overall went up a rank, down, or stayed same?

In [ ]:
s2 = moreStyleDriverSplitReportBaseDataframe(rb4cTop10, STAGE)
display(HTML(s2))

In [ ]:
getTablePNG(s2)

In [ ]:
!pwd

## Grab data over several stages

In [ ]:
##sketch - data grab

#Create a dataframe of stage times over several stages
rallydata = pd.DataFrame()

for stage in [1,2,3,4,5, 6]:
    _data = _get_timing( _get_data(stage), TIME)[TIME]
    _timing_data_long = _timing_long(_data)
    _timing_data_long.insert(0,'stage', stage)
    rallydata = pd.concat([rallydata, _timing_data_long], sort=False)